In [1]:
import pandas as pd
import numpy as np
import csv as csv
import matplotlib.pyplot as plt

import seaborn as sns
import re
import time
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import warnings
warnings.filterwarnings('ignore')

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor

import statsmodels.api as sm
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC, SVR
from sklearn.cluster import KMeans

from sklearn import metrics
from sklearn.metrics import r2_score,mean_absolute_error, mean_squared_error
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV
from os import path
from functools import reduce
import datetime

### Basic understanding of data

In [2]:
# Loading monthly price information from 1967 to 2020
# CUSIP number (one of the most important columns to join fundamental & price data) was introduced in 1967 only

monthly_stock_data = pd.read_csv(r'C:\Users\joshn\Downloads\MS in Data Science\Stock market analysis\Monthly stock data.csv')
monthly_stock_data['date'] = pd.to_datetime(monthly_stock_data['date'])
monthly_stock_data['Year'] = monthly_stock_data['date'].dt.year
monthly_stock_data['Month'] = monthly_stock_data['date'].dt.month

In [3]:
#there are 64 columns in the data
monthly_stock_data.shape

(4200298, 21)

In [4]:
len(monthly_stock_data['CUSIP'].unique())

33534

In [5]:
# I am not going to impute. First need to study each column
# monthly_stock_data = monthly_stock_data.fillna(0)

In [6]:
#Saving data of unique values into data frame
Num_unique_eachcol = pd.DataFrame(monthly_stock_data.apply(lambda x: x.nunique()))
Num_unique_eachcol = Num_unique_eachcol.reset_index()
#Num_unique_eachcol.columns = ['Column_name', 'Num_unique_values']
Num_unique_eachcol.rename(columns = {'index':'Column_name', 0: 'Num_unique_values'}, inplace = True)

In [7]:
# Calculating fill rate
null_df = pd.DataFrame(monthly_stock_data.isnull().sum()).reset_index()
null_df.rename(columns = {'index':'Column_name', 0:'Number_of_Null_values'}, inplace = True)
null_df['Fill_rate_percentage'] = ((94524 - null_df['Number_of_Null_values'])/94524)*100
# Just an alternate way of creating fill rate column
#for i in df2.index:
#    df2['Fill_rate_percentage'][i] = ((94524 - df2['Number_of_Null_values'][i])/94524)*100

# Creating a fill rate category
#if else statement for creating a new column condition based.np.where() is easier to use 
#null_df['Fill_rate_category'] = None0
#for i in null_df.index:
    #if null_df['Fill_rate_percentage'][i]<10:
        #null_df['Fill_rate_category'][i]='less_than_10'
    #else:
        #null_df['Fill_rate_category'][i]='btw_10_80'
    #if null_df['Fill_rate_percentage'][i] > 80:
        #null_df['Fill_rate_category'][i]='grt_than_80'
    #else:
        #null_df['Fill_rate_category']

#creating a new column using np.where(). Edditing the same code as above using np.where()
null_df['Fill_rate_category'] = np.where(null_df['Fill_rate_percentage']>10, 'btw_10_80', 'less_than_10')
null_df['Fill_rate_category'] = np.where(null_df['Fill_rate_percentage']>80, 'grt_than_80', null_df['Fill_rate_category'])               

In [8]:
#merging columns, inner join
merge_data = pd.merge(Num_unique_eachcol,
                      null_df[['Column_name','Fill_rate_percentage','Fill_rate_category']],
                      left_on = 'Column_name',
                      right_on = 'Column_name',
                      how = 'inner')

# A new csv file has been created by unique_values & fillrate.
merge_data.to_csv(r'C:\Users\joshn\Downloads\MS in Data Science\Stock market analysis\unique_values_fillrate.csv',index = False)

In [9]:
# Joining share code (type of stock/index/foreign company) into main data
share_code_data = pd.read_csv(r'C:\Users\joshn\Downloads\MS in Data Science\Stock market analysis\share code description.csv')

print(monthly_stock_data.shape)
monthly_stock_data = pd.merge(monthly_stock_data,
                              share_code_data,
                              left_on = 'SHRCD',
                              right_on = 'SHRCD',
                              how = 'left')
print(monthly_stock_data.shape)

(4200298, 21)
(4200298, 22)


In [10]:
print(monthly_stock_data['SHRCD_Description'].isnull().sum())
# Realized that SHRCD_Description has few nulls

# Based on testing few CUSIP's, figured out SHRCD_Description is null for every first row of CUSIP
# If a stock lists on Nov 2019, noticed Nov row SHRCD_Description is blank. 
monthly_stock_data = monthly_stock_data.sort_values(by=['date','CUSIP'],ascending = True)
monthly_stock_data['CUSIP_shift'] = monthly_stock_data['CUSIP'].shift(1)
print(monthly_stock_data[((monthly_stock_data['CUSIP'] != monthly_stock_data['CUSIP_shift']) &
                          (monthly_stock_data['SHRCD_Description'].isnull()))]['SHRCD_Description'].isnull().sum())

30089
30089


In [11]:
# So, I am back filling SHRCD_Description. The data is already in order
# monthly_stock_data['SHRCD_Description'] = monthly_stock_data['SHRCD_Description'].bfill()
# But I need to ensure the above code works correct under all circumstances
# As I realised if any company left market in a month then the 1st month data of the company will be null and it will be filled by next company SHRCD_Description
# So, to avoid such errors I decided to groupby SHRCD_Description with CUSIP and bfill the null values
monthly_stock_data['SHRCD_Description'] = monthly_stock_data.groupby('CUSIP')['SHRCD_Description'].transform(lambda v: v.bfill())
# Let's check distribution of SHRCD_Description now
monthly_stock_data['SHRCD_Description'].value_counts()

Normal Stock           3226976
Foreign Companies       346737
ETF                     247507
Close end funds         225669
REIT                     95782
Open end funds           39151
Trust                    14237
Foreign Trust             3175
Foreign mutual fund       1064
Name: SHRCD_Description, dtype: int64

In [12]:
monthly_stock_data = monthly_stock_data.sort_values(by=['date','CUSIP'],ascending = True)
cusip_shrcd_mapping = pd.DataFrame(monthly_stock_data.groupby('CUSIP')['SHRCD_Description'].nunique()).reset_index()
cusip_shrcd_mapping.columns = ['CUSIP', '# Unique SHRCD_Descriptions']
cusip_shrcd_mapping.sort_values(by = ['# Unique SHRCD_Descriptions'], ascending = False)
cusip_shrcd_mapping['# Unique SHRCD_Descriptions'].value_counts()
# Almost all companies have only 1 SHRCD description. There are few that were one REIT/Trust and later transformed to Normal stock
# Let's just ignore them

1    33173
2      347
3       14
Name: # Unique SHRCD_Descriptions, dtype: int64

In [13]:
shrcd_to_consider = ['Normal Stock','Foreign Companies','Probably_Normal_stock']

In [14]:
# Let us calculate M.Cap of each stock
monthly_stock_data['Mcap'] = monthly_stock_data['PRC'].abs()*monthly_stock_data['SHROUT']*1000

### Figuring out level of data, primary key

In [15]:
# Figuring out level of data
# monthly_stock_data['PERMNO_str'] = df['PERMNO'].astype(str) - No need to create a new column
monthly_stock_data['possible_level_of_data'] = monthly_stock_data['PERMNO'].astype(str) + monthly_stock_data['date'].astype(str)
len(monthly_stock_data['possible_level_of_data'].unique()) #unfortunately permno and date are not matching so i am creating a new dataframe

#important tips for variable naming
#A variable name should be as small as possible
#only 1st letter should be capital
#the most imp characteristics of a variable name should be within the 1st 7 to 8 letters

4200298

In [16]:
# Writting a for loop for the best 5 possibilities to identify as level of data. running iteration for a null data frame and adding new rows accordingly.
level_of_data_exploration = pd.DataFrame(None) # better to have a null df, It will be easy to use during iterations.
for i in ['NCUSIP','TICKER','PERMCO','CUSIP','PERMNO']: # i should be used wherever the change of data is required.
    monthly_stock_data['possible_level_of_data'] = monthly_stock_data[i].astype(str) + monthly_stock_data['date'].astype(str)
    x1 = i
    x2 = len(monthly_stock_data['possible_level_of_data'].unique())
    x3 = len(monthly_stock_data)
#    print(x1)
#    print(x2)
#    print(x3)
    level_of_data_exploration_iter = pd.DataFrame([[x1,x2,x3]], columns=['Column_name', 'unique_values_combined_date', 'rows_in_data'])
    level_of_data_exploration = pd.concat([level_of_data_exploration, level_of_data_exploration_iter], axis = 0)
    
# 3 things to remember in a for loop
# 1) Using iterator (i) in the right places
# 2) Use print statements to debug
# 3) Sometimes, i may need to initialize a series or dataframe before the for loop

level_of_data_exploration
# The data is unique at both PERMNO, date & CUSIP, date

,Column_name,unique_values_combined_date,rows_in_data
0,NCUSIP,4142382,4200298
0,TICKER,4075832,4200298
0,PERMCO,3930704,4200298
0,CUSIP,4200298,4200298
0,PERMNO,4200298,4200298


In [17]:
print(len(monthly_stock_data['CUSIP'].unique()))
print(len(monthly_stock_data['NCUSIP'].unique()))

cusip_ncusip_map = pd.DataFrame(monthly_stock_data.groupby('CUSIP')['NCUSIP'].nunique()).reset_index()
cusip_ncusip_map.columns = ['CUSIP', 'Num_unique_NCUSIP']
cusip_ncusip_map = cusip_ncusip_map.sort_values(by=['Num_unique_NCUSIP'],ascending = False)
cusip_ncusip_map[0:5]
# monthly_stock_data[monthly_stock_data['CUSIP'] == '74347W14'].to_csv(r'C:\Users\joshn\Downloads\MS in Data Science\Stock market analysis\cusip_data.csv')
# Realised that NCUSIP can change with time. CUSIP is actually the most recent NCUSIP for every stock

33534
46267


,CUSIP,Num_unique_NCUSIP
21998,68235H30,13
33529,Y8897Y18,12
15368,45857P80,10
18528,55357010,10
24209,74347W14,10


In [18]:
# Calculating 6 digit CUSIP. 
# Ideally, CRSP data should be unique at CUSIP_6digit & date
# The first 6 digits of a CUSIP uniquely identify a company
# The last 2 digits of CUSIP indicate Share class or some name change, ticker change, restructuring

monthly_stock_data['cusip_6digit'] = monthly_stock_data['CUSIP'].str[0:6]

# I am also gonna create a columns called cusip_8digit that's literally the original CUSIP
monthly_stock_data['cusip_8digit'] = monthly_stock_data['CUSIP']
# CRSP actaully has 8 digit CUSIP by default (unlike compusat which provides 9 digit cusip by default)

In [19]:
# However, before I work with CUSIP_6 digit, let's understand if there's instances of 1 CUSIP_6digit mapping to multiple CUSIP_8digits

# A Cusip_6digit can actaully map to multiple cusip_8digits indeed
# Expected reasons are
# 1) Index funds - because components of an index change continuously, Indices get multiple CUSIP's
# Solution: not ideal, but analysis is not focused on index funds
# 2) Some companies have multiple share classes. ClassA and ClassB shares will have same CUSIP_6digit but different CUSIP_8digit
# Solution: need to pick just one class share for every CUSIP_6digit. Have to do this

In [20]:
# Data indicates there's actually quite a few companies with multiple classes
monthly_stock_data['SHRCLS'].value_counts()
# Need to account for this if I want to use cusip_6digit & date as level of data
# Moreover, need to make sure one class can appear only once in data. 
# Can't have 3 rows for Google just because google happens to have classA, classB, classC

A    163783
B     44410
C      1507
D       908
L       766
H       689
E       481
N       277
V       260
1       136
P       119
U        92
G        52
Z        34
T        10
Name: SHRCLS, dtype: int64

In [21]:
# Let's see if there's any other reasons for multiple mapping (apart from the 2 reasons already mentioned)

cusip_shrcls_unique = pd.DataFrame(monthly_stock_data.groupby('cusip_6digit')['SHRCLS'].nunique()).reset_index()
cusip_shrcls_unique.columns = ['cusip_6digit' , 'Num_unique_shrcls']
cusip_shrcls_unique['Num_unique_shrcls'].value_counts()
# Getting CUSIP_6digits of all the people with more than 1 share class
cusip_6digit_multiple_shrcls = cusip_shrcls_unique[cusip_shrcls_unique['Num_unique_shrcls'] > 1]['cusip_6digit']

# Let's exclude indices & stocks with more than 1 shareclass (known 2 reasons)
comp_btw_cusip6_cusip8_multiple_values = pd.DataFrame(monthly_stock_data[(monthly_stock_data['SHRCD_Description'].isin(shrcd_to_consider)) & 
                                                                        (~monthly_stock_data['cusip_6digit'].isin(cusip_6digit_multiple_shrcls))]
                                                      .groupby('cusip_6digit')['cusip_8digit'].nunique()).reset_index()
comp_btw_cusip6_cusip8_multiple_values.columns = ['cusip_6digit', 'Num_unique_cusip_8digit']
comp_btw_cusip6_cusip8_multiple_values = comp_btw_cusip6_cusip8_multiple_values.sort_values(by=['Num_unique_cusip_8digit'],
                                                                                            ascending = False)
print(comp_btw_cusip6_cusip8_multiple_values['Num_unique_cusip_8digit'].value_counts())
# Realized that when a cusip_6digit is a normal stock & has just 1 unique share class associated with it, (continued below)
# there's 1-1 mapping in most of cases between CUSIP (6 digit) & CUSIP (8 digit)

# For investigation of some of the weird cases, let's pick few companies that are still active in 2020
cusip_6digit_2020_active = monthly_stock_data[monthly_stock_data['Year'] == 2020]['cusip_6digit']
comp_btw_cusip6_cusip8_multiple_values[comp_btw_cusip6_cusip8_multiple_values['cusip_6digit'].isin(cusip_6digit_2020_active)][0:5]

# Upon further investigation (code below), last 2 digits of CUSIP can also change during 2 additional scenarios
# 3) Restructuring. If a company went through restructruing & emerged, I think CUSIP bureau automatically assigns different CUSIP number with same first 6 digits
# Solution: Don't need to think of this a major problem. See comments below
# 4) There are some companies like Google that change name to Alphabet (and manually apply for a new CUSIP)
# Solution: Don't need to think of this a major problem. See comments below

# Realized both (3) and (4) are weird folks who manually applied for a new CUSIP after a name, ticker change
# Don't have to worry about them
# If a company used CUSIP g5433m34 from 1980-2000 & suddenly changed to g5433m67 (same first 6 digits), it's still same company
# Just because google changed last 2 digits of cusip after becoming alphabet, doesn't mean I can't calculate last_5Y % change

# To conclude, there are 4 reasons why 1 CUSIP_6digit can map to multiple CUSIP_8digits
# But there's only type of instance that I need to specifically correct for: a company having multiple classes

1    26703
2      266
3        4
4        1
5        1
Name: Num_unique_cusip_8digit, dtype: int64


,cusip_6digit,Num_unique_cusip_8digit
270,003881,3
19407,747906,3
13578,500472,3
16530,640671,2
237,002896,2


In [22]:
# In cases when both classA & classB are present, let's see which class generally has better volumes

monthly_stock_data['volume_price'] = monthly_stock_data['VOL'] * abs(monthly_stock_data['PRC'])
monthly_stock_data['cusip_6digit_date'] = monthly_stock_data['cusip_6digit'].astype(str) + monthly_stock_data['date'].astype(str)

cusip_6digit_shrA_vol = monthly_stock_data[monthly_stock_data['SHRCLS'] == 'A'][['cusip_6digit','date','cusip_6digit_date', 
                                                                                 'volume_price']]
cusip_6digit_shrA_vol.rename(columns = {'volume_price' : 'volume_price_shrA'},inplace = True)

cusip_6digit_shrB_vol = monthly_stock_data[monthly_stock_data['SHRCLS'] == 'B'][['cusip_6digit','date','cusip_6digit_date','volume_price']]
cusip_6digit_shrB_vol.rename(columns = {'volume_price' : 'volume_price_shrB'},inplace = True)

#print(cusip_6digit_shrA_vol.shape)
#print(cusip_6digit_shrB_vol.shape)
cusip_6digit_shrAB_vol = pd.merge(cusip_6digit_shrA_vol[['cusip_6digit','date','volume_price_shrA']],
                                  cusip_6digit_shrB_vol[['cusip_6digit','date','volume_price_shrB']],
                                  left_on = ['cusip_6digit','date'],
                                  right_on = ['cusip_6digit','date'],
                                  how = 'inner')
#print(cusip_6digit_shrAB_vol.shape)

cusip_6digit_shrAB_vol['volA_div_volB'] = cusip_6digit_shrAB_vol['volume_price_shrA']/cusip_6digit_shrAB_vol['volume_price_shrB']
print(cusip_6digit_shrAB_vol['volA_div_volB'].describe())
# Realized that when both classA & classB are available, classA dollar volume is generally larger

count    2.330000e+04
mean              inf
std               NaN
min      0.000000e+00
25%      5.241846e-01
50%      3.572992e+00
75%      4.605866e+01
max               inf
Name: volA_div_volB, dtype: float64


In [23]:
# Note that there are quite a few stocks where only classB is present in a monthly row (no class A)
# Can't just delete all rows with SHRCD != A
monthly_stock_data[monthly_stock_data['cusip_6digit_date'].isin(cusip_6digit_shrB_vol['cusip_6digit_date'])]['SHRCLS'].value_counts()

B    44410
A    23744
C      647
L       86
Name: SHRCLS, dtype: int64

In [24]:
first_shrcls_that_appears = monthly_stock_data[~monthly_stock_data['SHRCLS'].isnull()][['cusip_6digit', 
                            'date', 'SHRCLS']].sort_values(by = ['cusip_6digit', 'date', 
                                                                 'SHRCLS'], 
                                                           ascending = True).drop_duplicates(subset = ['cusip_6digit'])
first_shrcls_that_appears['SHRCLS'].value_counts()[0:3]
# Realized that 'A' is often the first class that appears for every stock

A    1728
B     183
D       8
Name: SHRCLS, dtype: int64

In [25]:
# Interestingly, noticed that even when B is the first listed stock, volume of A is larger
print(cusip_6digit_shrAB_vol[cusip_6digit_shrAB_vol['cusip_6digit'].isin(first_shrcls_that_appears
                                                                         [first_shrcls_that_appears['SHRCLS'] == 'B']
                                                                         ['cusip_6digit'])]['volA_div_volB'].describe())

count    4535.000000
mean             inf
std              NaN
min         0.000000
25%         1.219236
50%         8.470588
75%        58.974251
max              inf
Name: volA_div_volB, dtype: float64


In [26]:
# When a company has listed classB first & classA later, let's see how late classA appears on market

first_date_of_shrclsA = monthly_stock_data[monthly_stock_data['SHRCLS'] == 'A'][['cusip_6digit','date']].sort_values(by = ['cusip_6digit','date'],
                                                                    ascending = True).drop_duplicates(subset = ['cusip_6digit'])
first_date_of_shrclsA.columns = ['cusip_6digit','first_date_of_shrclsA']

first_date_of_shrclsB = monthly_stock_data[monthly_stock_data['SHRCLS'] == 'B'][['cusip_6digit','date']].sort_values(by = ['cusip_6digit','date'],
                                                                    ascending = True).drop_duplicates(subset = ['cusip_6digit'])
first_date_of_shrclsB.columns = ['cusip_6digit','first_date_of_shrclsB']

first_date_of_shrclsAB = pd.merge(first_date_of_shrclsA,
                                  first_date_of_shrclsB,
                                  left_on = ['cusip_6digit'],
                                  right_on = ['cusip_6digit'],
                                  how = 'inner')

first_date_of_shrclsAB['gap_btw_shrclsA_shrclsB'] = pd.to_datetime(first_date_of_shrclsAB['first_date_of_shrclsA']) - pd.to_datetime(first_date_of_shrclsAB['first_date_of_shrclsB'])
first_date_of_shrclsAB['gap_btw_shrclsA_shrclsB'] = first_date_of_shrclsAB['gap_btw_shrclsA_shrclsB'].dt.days

first_date_of_shrclsAB[first_date_of_shrclsAB['cusip_6digit'].isin(first_shrcls_that_appears
                                                                   [first_shrcls_that_appears['SHRCLS'] == 'B']['cusip_6digit'])]['gap_btw_shrclsA_shrclsB'].describe()
# Very interesting to note that even when classB is first listed, in almost half cases classA stock gets listed in 60 days

count       29.000000
mean       927.862069
std       2158.298151
min         28.000000
25%         31.000000
50%         60.000000
75%        700.000000
max      10836.000000
Name: gap_btw_shrclsA_shrclsB, dtype: float64

In [27]:
# Summarizing our research on different classes
# 1) when both stockA & stockB are listed, volume of stockA is higher
# 2) StockA also is the class that often appears first on market
# 3) Even for companies where stockB appears first, volume of classA (which is listed later) is larger
# 4) Further, in cases where classB is listed first, classA often appears very soon

# Based on the learnings above, if a company (CUSIP_6digit) has classA in data, let's just take only classA rows 
# (irrespective of whether the company also has classB/classC and listed them before/after etc.)

# While the above insights are obtained focusing on classA & classB only, let's apply the same logic to other classes like C,D also

In [28]:
best_shrclas_every_cusip6digit = monthly_stock_data[~monthly_stock_data['SHRCLS'].
                                                    isnull()][['cusip_6digit', 
                                                               'SHRCLS']].sort_values(by = ['cusip_6digit', 'SHRCLS'], 
                                                                                      ascending = True).drop_duplicates(subset = ['cusip_6digit'])
# Getting first alphabet share class for every cusip 6digit
best_shrclas_every_cusip6digit.columns = ['cusip_6digit','first_alphabet_shrcls']


print(monthly_stock_data.shape)
monthly_stock_data = pd.merge(monthly_stock_data,
                              best_shrclas_every_cusip6digit,
                              left_on = ['cusip_6digit'],
                              right_on = ['cusip_6digit'],
                              how = 'left')
print(monthly_stock_data.shape)


print(monthly_stock_data.shape)
# Subsetting only for rows where shareclass is matching with first alphabet shareclass
monthly_stock_data = monthly_stock_data[(monthly_stock_data['SHRCLS'].isnull()) |
                                        (monthly_stock_data['SHRCLS'] == monthly_stock_data['first_alphabet_shrcls'])]
print(monthly_stock_data.shape)
# Finally, figured out how to deal with stocks that have multiple classes (same CUSIP_6digit appearing twice in a month)

(4200298, 29)
(4200298, 30)
(4200298, 30)
(4172195, 30)


In [29]:
print(monthly_stock_data.groupby('cusip_6digit_date')['SHRCLS'].nunique().value_counts())
# Based on our effort so far, 1 cusip_6digit & date can have only 1 shareclass

# But realized there are even weirder companies that have 3 share class 'A' in the same month
print(monthly_stock_data[monthly_stock_data['SHRCLS'] == 'A']['cusip_6digit_date'].value_counts())

# Investigated them and realized they are just 'weird'!
imp_cols_for_investigation = ['date','cusip_6digit','cusip_8digit','TICKER','COMNAM','SHRCLS','RET','Mcap']
monthly_stock_data[monthly_stock_data['cusip_6digit_date'] == 'G5480U2017-09-29'][imp_cols_for_investigation]

0    3753645
1     184976
Name: SHRCLS, dtype: int64
5312292020-08-31    3
5312292017-01-31    3
87924V1998-09-30    3
5312292019-11-29    3
87924V1999-01-29    3
                   ..
8788952006-09-29    1
7059042005-04-29    1
92645B2020-08-31    1
4835481977-03-31    1
3131482006-04-28    1
Name: cusip_6digit_date, Length: 163338, dtype: int64


,date,cusip_6digit,cusip_8digit,TICKER,COMNAM,SHRCLS,RET,Mcap
3902241,2017-09-29,G5480U,G5480U10,LBTYA,LIBERTY GLOBAL PLC,A,-0.002647,7.524426e+09
3902244,2017-09-29,G5480U,G5480U13,LILA,LIBERTY GLOBAL PLC,A,-0.086505,1.149936e+09


In [30]:
# But, is it possible for CUSIP (6digit) to appear twice in same month for normal stocks that don't have any SHRCLS filled in?

# when share class is not available at all, there are very few instances of one CUSIP_6digit having 2 rows in the same month
# There are 3 possible reasons for this
# 1) There are some 'great' companies like liberty media which have 3 classA stocks within a month
# 2) CRSP didn't properly SHRCLS info for such rows. The last 2 digits are different because that stock had classA, classB in same month
# 3) Some folks changed their name (& obtained different CUSIP with different last 2 digits) in the middle of month
#    So their CUSIP_6digit appears twice in the same month

same_cusip_date_but_multiplerows = pd.DataFrame(monthly_stock_data[monthly_stock_data['SHRCD_Description'].isin(shrcd_to_consider)]['cusip_6digit_date'].value_counts()).reset_index()
same_cusip_date_but_multiplerows.columns = ['cusip_6digit_date','Num_rows']
print(same_cusip_date_but_multiplerows['Num_rows'].value_counts())
# In the below value counts noticed there are nearly 9000 rows where data is not unique at cusip_6digit_date level
print(len(same_cusip_date_but_multiplerows[same_cusip_date_but_multiplerows['Num_rows'] > 1]['cusip_6digit_date'].astype(str).str[0:6].unique()))
# Fortunately, this problem is only for 286 companies

# Let's just sort the data for cusip_6digit, cusip_8digit, date and drop duplicates 
print(monthly_stock_data.shape)
monthly_stock_data = monthly_stock_data.sort_values(by = ['cusip_6digit', 'cusip_8digit', 
                                                          'date']).drop_duplicates(subset = ['cusip_6digit', 'date'])
print(monthly_stock_data.shape)
# Removed nearly 200k rows from overall data. Almost 190k rows have been removed because of index funds
# For normal stocks removed only 10k rows

1    3527764
2       8880
3        145
4         20
5          2
6          1
Name: Num_rows, dtype: int64
286
(4172195, 30)
(3938621, 30)


In [31]:
# Checking if one unique combination of Cusip_6digit & date has only one unique Ret or not
# using nunique

num_unique_ret_for_every_cusip6date = pd.DataFrame(monthly_stock_data.groupby('cusip_6digit_date')['RET'].nunique()).reset_index()
num_unique_ret_for_every_cusip6date.columns = ['cusip_6digit_date', 'Num_unique_RET']
num_unique_ret_for_every_cusip6date['Num_unique_RET'].value_counts()
# One cusip_6digit_date has only  1 unique RET

1    3867329
0      71292
Name: Num_unique_RET, dtype: int64

In [32]:
# The data is finally unique at CUSIP_6digit & date. 
# Assuming that if 2 rows have different CUSIP_6 digits, they are totally different companies
# But saw a lot of companies where 2 different CUSIP's are actually the same company
# But wondering if it's possible for company to have multiple CUSIP_6digits?

print(monthly_stock_data.groupby('COMNAM')['cusip_6digit'].nunique().value_counts())
# Looks like for every company name, there is a unique CUSIP_6digit

1     40008
2       188
3         5
5         4
4         2
8         2
9         1
12        1
Name: cusip_6digit, dtype: int64


In [33]:
# But what about ticker?
print(monthly_stock_data.groupby('TICKER')['cusip_6digit'].nunique().value_counts())
# Wow. One particular ticker had 9 different CUSIP_6digits associated with it

# I understand that different companies can use the same ticker across time. But 9 different companies using the same ticker?
# That must be one hell of a ticker

print(len(monthly_stock_data['COMNAM'].unique()))
print(len(monthly_stock_data['TICKER'].unique()))
# Noticed that there are only 29,100 unique ticker's in the entire data
# The number of unique values in ticker is significantly lesser than number of unique values in CUSIP, PERMNO, etc.
# May be I can just assume any 2 cusip_6digits with same ticker as the same company?

1    22482
2     4624
3     1436
4      408
5      106
6       33
7        9
9        1
Name: cusip_6digit, dtype: int64
40212
29100


In [34]:
# But turns out sometimes, 2 Cusip_6digits associated with same ticker can be very different companies

monthly_stock_data[monthly_stock_data['TICKER'] == 'G'][['COMNAM']].value_counts()
# Clearly, I can't just rely on ticker to identify each company uniquely
# The trick to identify the cases tickers like 'GM' where 2 Cusip_6 digits are legit the same General Motors
# While avoiding false positives like 'G' where multiple Cusip_6 digits that are associated with it are different companies

# This is really important because I intend to calculate prior 1Y return, 5Y return etc. columns for each company later
# If code doesn't recognize the fact that 2 CUSIP_6digits belong to the same company, then I can't calculate those columns

COMNAM             
GREYHOUND CORP         280
GILLETTE CO            170
GENPACT LTD            161
GREYHOUND DIAL CORP     12
dtype: int64

In [35]:
# I think compustat data could be very relevant here

# Already know from documentation that the CUSIP column in both CRSP & Compustat databases is the most recent CUSIP for every stock
# But discovered that the phrase 'most recent' has very different meaning in both the data sources
# In Compustat, the CUSIP is truly ther final CUSIP that appears for every stock
# For example, if GM went bankrupt in 2009 & came back in 2010, compustat has put the 2021 GM CUSIP for all GM rows since 1967
# In sharp contrast, CRSP takes the fact that a major corporate event (bankruptcy/restructuring/merger) took place in 2009
# In CRSP's world, the 'old' GM stopped trading in 2009. The GM that came back in 2010 is another new company
# There's countless cases like this: General motors, Kmart, Abercrombie etc.
# Summary: The CUSIP in CRSP not truly the real most recent CUSIP. It is the most recent CUSIP of the companies current shape

In [36]:
# I think the difference (explained above) block between the 2 databases could be really helpful in figuring out interlinked cusips

# For example, let's say GM had cusip x from 1967 - 2009, cusip y from 2010-2020
# Just by using monthly stock data, there is no way to know that x & y belong to GM
# Once again: Not every cusip associated with same ticker are the same company
# But in compustat data, all GM rows from 1967 onwards have cusip y
# So, by joining both data sets on something like ticker, date, month, I can probably map x to y (GMJan1980 mapped to GMJan1980)
# Once I have mapped all such interlinked cusips, I will be able to identify that x and y are indeed interlinked cusips

In [37]:
# Loading prepared fundamental data
comp_data = pd.read_csv(r'C:\Users\joshn\Downloads\MS in Data Science\Stock market analysis\fundemental_data_prepared.csv')
comp_data['cusip_6digit'] = comp_data['cusip'].astype(str).str[0:6]
comp_data['cusip_8digit'] = comp_data['cusip'].astype(str).str[0:8]

comp_data.rename(columns = {'naics':'NAICS'}, inplace = True)

In [38]:
# Discovered it won't be easy to join CRSP & Compustat data

print(monthly_stock_data.shape)
print(monthly_stock_data[~monthly_stock_data['cusip_6digit'].isin(comp_data['cusip_6digit'])].shape)
# Lot of cusip_6digits on CRSP side which just don't appear anywhere in Compustat side

# Noticed that there are few cases where cusip_6digit match is not found but there is a company name match
print(monthly_stock_data[((monthly_stock_data['COMNAM'].isin(comp_data['company_name'])) &
                   (~monthly_stock_data['cusip_6digit'].isin(comp_data['cusip_6digit'])))].shape)

# There's a ton of cases where cusip_6digit match is not found but there is a ticker match
print(monthly_stock_data[((monthly_stock_data['TICKER'].isin(comp_data['ticker'])) &
                   (~monthly_stock_data['cusip_6digit'].isin(comp_data['cusip_6digit'])))].shape)
# Ofcourse, some of these ticker matches are going to completely different companies
# But, there's going to be 2 types of legit cases
# 1) Cases like GM where CUSIP x will not be found in Compustat but is actually linked to y (that's there in CRSP too)
# 2) realized  in some cases CUSIP a is actually CUSIP b on the compustat side (continued below)
# But if all rows in CRSP belonging are already coded a, then I guess I am not gonna find any interlinks btw. a & some other CUSIP
# However, the fact that a & b are linked is gonna help me when joining CRSP & Compustat (eventually)

(3938621, 30)
(438619, 30)
(35142, 30)
(149654, 30)


In [39]:
# let's investigate few big companies that don't have a match on the compustat side

monthly_stock_data[(monthly_stock_data['Year'] == 1973) & 
                   (~monthly_stock_data['cusip_6digit'].isin(comp_data
                  ['cusip_6digit']))][imp_cols_for_investigation].sort_values(by = ['Mcap'],ascending = False)[0:5]

# GM is not being matched? wow!

,date,cusip_6digit,cusip_8digit,TICKER,COMNAM,SHRCLS,RET,Mcap
181888,1973-01-31,370442,37044210,GM,GENERAL MOTORS CORP,NaN,-0.047766,2.208755e+10
187727,1973-02-28,370442,37044210,GM,GENERAL MOTORS CORP,NaN,-0.037540,2.101534e+10
193567,1973-03-30,370442,37044210,GM,GENERAL MOTORS CORP,NaN,-0.017007,2.065794e+10
199393,1973-04-30,370442,37044210,GM,GENERAL MOTORS CORP,NaN,-0.012111,2.040775e+10
216683,1973-07-31,370442,37044210,GM,GENERAL MOTORS CORP,NaN,0.016917,1.933554e+10


In [40]:
monthly_stock_data[monthly_stock_data['cusip_6digit'] == '370442'][imp_cols_for_investigation]
# Last month of GM trading is June 2009? That doesn't make sense. GM is currently listed on stock market

,date,cusip_6digit,cusip_8digit,TICKER,COMNAM,SHRCLS,RET,Mcap
850,1967-01-31,370442,37044210,GM,GENERAL MOTORS CORP,NaN,0.140417,2.146163e+10
3067,1967-02-28,370442,37044210,GM,GENERAL MOTORS CORP,NaN,-0.025291,2.067602e+10
5286,1967-03-31,370442,37044210,GM,GENERAL MOTORS CORP,NaN,0.053541,2.178302e+10
7510,1967-04-28,370442,37044210,GM,GENERAL MOTORS CORP,NaN,0.134426,2.471123e+10
9736,1967-05-31,370442,37044210,GM,GENERAL MOTORS CORP,NaN,-0.079769,2.249722e+10
...,...,...,...,...,...,...,...,...
3176438,2009-02-27,370442,37044210,GM,GENERAL MOTORS CORP,NaN,-0.252492,1.373542e+09
3183305,2009-03-31,370442,37044210,GM,GENERAL MOTORS CORP,NaN,-0.137778,1.184374e+09
3190134,2009-04-30,370442,37044210,GM,GENERAL MOTORS CORP,NaN,-0.010309,1.172164e+09
3196925,2009-05-29,370442,37044210,GM,GENERAL MOTORS CORP,NaN,-0.609375,4.579215e+08


In [41]:
# Interestingly, GM does appear in compustat data. But GM has a very different CUSIP number
# Found that GM's real current CUSIP number is actually 37045V10 by googling

comp_data[comp_data['cusip_8digit'] == '37045V10'][['company_name', 'quarter_end_date', 'revenue']]
# Verified this cusip is actually GM by tallying reveune numbers
# In Compustat data, GM appeasr as one continuous stock that has been listed on stock market since 1967
# So, what is going on?

,company_name,quarter_end_date,revenue
236107,GENERAL MOTORS CO,1967-03-31,4854.297
236108,GENERAL MOTORS CO,1967-06-30,5561.500
236109,GENERAL MOTORS CO,1967-09-30,3776.400
236110,GENERAL MOTORS CO,1967-12-31,5834.098
236111,GENERAL MOTORS CO,1968-03-31,5370.500
...,...,...,...
236318,GENERAL MOTORS CO,2019-12-31,30826.000
236319,GENERAL MOTORS CO,2020-03-31,32709.000
236320,GENERAL MOTORS CO,2020-06-30,16778.000
236321,GENERAL MOTORS CO,2020-09-30,35480.000


In [42]:
# As a matter of fact, I can see few GM rows even in CRSP data for the CUSIP: 37045V10
monthly_stock_data[monthly_stock_data['CUSIP'] == '37045V10'][imp_cols_for_investigation]
# The earliest month this CUSIP appears is Oct 2010
# Why are GM rows under 2 different tickers in CRSP data? One CUSIP appears from 1967 to 2009 & there's another CUSIP from 2010

# This GM case perfectly illustrates the difference between CRSP & Compustat

,date,cusip_6digit,cusip_8digit,TICKER,COMNAM,SHRCLS,RET,Mcap
3311647,2010-10-29,37045V,37045V10,NaN,NaN,NaN,NaN,NaN
3318415,2010-11-30,37045V,37045V10,GM,GENERAL MOTORS CO,NaN,C,5.130000e+10
3325192,2010-12-31,37045V,37045V10,GM,GENERAL MOTORS CO,NaN,0.077778,5.793286e+10
3331970,2011-01-31,37045V,37045V10,GM,GENERAL MOTORS CO,NaN,-0.010038,5.735133e+10
3338747,2011-02-28,37045V,37045V10,GM,GENERAL MOTORS CO,NaN,-0.081118,5.269910e+10
...,...,...,...,...,...,...,...,...
4163355,2020-08-31,37045V,37045V10,GM,GENERAL MOTORS CO,NaN,0.190438,4.240340e+10
4171197,2020-09-30,37045V,37045V10,GM,GENERAL MOTORS CO,NaN,-0.001350,4.234616e+10
4179068,2020-10-30,37045V,37045V10,GM,GENERAL MOTORS CO,NaN,0.166948,4.941578e+10
4187025,2020-11-30,37045V,37045V10,GM,GENERAL MOTORS CO,NaN,0.269621,6.274854e+10


In [43]:
# Checking level of fundamental data
# Fundamental data is nearly unique at cusip_6digit, qtr & yr 

print(comp_data.shape)
comp_data['cusip_6digit_date'] = comp_data['cusip_6digit'].astype(str) + comp_data['fiscal_quarter'].astype(str) + comp_data['fiscal_year'].astype(str)
print(len(comp_data['cusip_6digit_date'].unique()))
# Fundamental data is perfectly unique at cusip_8digit, qtr & yr 
comp_data['cusip_8digit_date'] = comp_data['cusip_8digit'].astype(str) + comp_data['fiscal_quarter'].astype(str) + comp_data['fiscal_year'].astype(str)
print(len(comp_data['cusip_8digit_date'].unique()))

comp_data['Month'] = pd.to_datetime(comp_data['quarter_end_date']).dt.month
comp_data['Year'] = pd.to_datetime(comp_data['quarter_end_date']).dt.year

(1828155, 64)
1755431
1828155


In [44]:
# Before exploring the use of ticker, there's something unique about TICKER column in compustat
# If the same ticker (Ex: 'AB') has been used by 4 different companies, Compustat has set tickers to AB, AB.1, AB.2, AB.3

# There's only one company with ticker 'AB' in compustat data
print(comp_data[comp_data['ticker']== 'AB']['company_name'].value_counts())
# Ticker of AMBAC Industries which also used to use the ticker AB, was coded as AB.1
print(comp_data[comp_data['ticker']== 'AB.1']['company_name'].value_counts())

# Need to correct cases like this before exploring ticker join. There's lot of such cases
print(comp_data[comp_data['ticker'].astype(str).str[-2:-1] == '.']['ticker'].astype(str).str[-2:].value_counts()[0:5])

# Creating a new ticker_mod column that will be using to explore join
#comp_data['ticker_mod'] = np.where(comp_data['ticker'].astype(str).str[-2:-1] == '.', 
#                                   comp_data['ticker'].astype(str).str[: -2], comp_data['ticker'])

# Rather than just removing '.' if it occurs in last but one digit, let's remove any characters that occur after a '.'
comp_data['ticker_mod'] = comp_data['ticker'].str.split('.').str[0]
comp_data['ticker_mod'] = comp_data['ticker_mod'].str.upper()

# Now all the companies that have used 'AB' across histroy are coded as 'AB'
print(comp_data[comp_data['ticker_mod'] == 'AB']['company_name'].value_counts())

ALLIANCEBERNSTEIN HOLDING LP    136
Name: company_name, dtype: int64
AMBAC INDUSTRIES INC    45
Name: company_name, dtype: int64
.1    164704
.2     57659
.3     17066
.Z      8354
.A      5055
Name: ticker, dtype: int64
ALLIANCEBERNSTEIN HOLDING LP    136
ASBESTOS CORP LTD               120
ALEX BROWN INC                   50
AMBAC INDUSTRIES INC             45
ABI AMERICAN BUSINESSPHONE       20
Name: company_name, dtype: int64


In [45]:
# Creating a new ticker_mod column in CRSP that will use to explore join
monthly_stock_data['ticker_mod'] = monthly_stock_data['TICKER'].str.split('.').str[0]
monthly_stock_data['ticker_mod'] = monthly_stock_data['ticker_mod'].str.upper()

In [46]:
# Wondering, if it's possible for 1 ticker to appear twice in each quarter
comp_data['ticker_qtr'] = comp_data['ticker_mod'].astype(str) + comp_data['Year'].astype(str) + comp_data['Month'].astype(str)
comp_ticker_qtr_repeat = pd.DataFrame(comp_data.groupby('ticker_qtr')['cusip_6digit'].nunique()).reset_index()
comp_ticker_qtr_repeat.columns = ['ticker_qtr', '# rows']
print(comp_ticker_qtr_repeat['# rows'].value_counts())
comp_ticker_qtr_problematic = comp_ticker_qtr_repeat[comp_ticker_qtr_repeat['# rows'] > 1]['ticker_qtr']

# Creating similar ticker_qtr column in CRSP data too
monthly_stock_data['ticker_qtr'] = monthly_stock_data['ticker_mod'].astype(str) + monthly_stock_data['Year'].astype(str) + monthly_stock_data['Month'].astype(str)
crsp_ticker_qtr_repeat = pd.DataFrame(monthly_stock_data.groupby('ticker_qtr')['cusip_6digit'].nunique()).reset_index()
crsp_ticker_qtr_repeat.columns = ['ticker_qtr', '# rows']
print(crsp_ticker_qtr_repeat['# rows'].value_counts())
crsp_ticker_qtr_problematic = crsp_ticker_qtr_repeat[crsp_ticker_qtr_repeat['# rows'] > 1]['ticker_qtr']

# Compiling a full list of problematic ticker qtr combinations
problematic_ticker_qtr = list(comp_ticker_qtr_problematic) + list(crsp_ticker_qtr_problematic)

1    1755079
2      34098
3       1124
4         34
Name: # rows, dtype: int64
1       3832744
2          2497
173           8
223           8
43            8
         ...   
34            1
197           1
35            1
38            1
3041          1
Name: # rows, Length: 230, dtype: int64


In [47]:
# Planning to use ticker on both sides to figure out additional matches
# The idea is that if 2 tickers are in the same period of time, the matches are same company
finding_interlinked_cusips = pd.merge(monthly_stock_data[~monthly_stock_data['NAICS'].isnull()][
                                      ['Month','Year','NAICS','cusip_6digit','ticker_mod','COMNAM','Mcap']],
                                      comp_data[~comp_data['ticker_qtr'].isin(problematic_ticker_qtr)][
                                          ['Month','Year','NAICS','gvkey','cusip_6digit','ticker_mod','company_name','Mcap']],
                                      left_on = ['Month','Year','ticker_mod','NAICS'],
                                      right_on = ['Month','Year','ticker_mod','NAICS'],
                                      how = 'inner')
# Why I am using NAICS code too in this join? (along with ticker's)
# For example, found that ABI american business phones (telecom stock) used to have a ticker AB. This company got acquired 1987
# AllianceBernstien (finance company) adopted the ticker 'AB' in 2000 after acquiring Bernstien
# Alliance used to be listed under the name Alliance capital with 'AC' originally in 1980's
# I think using NAICS code in this join ensures that a lot of mismatches are weeded out
# It's extremely unlikely 2 different companies with same NAICS code used same ticker in the same period of time
# Ofcourse, NAICS codes in monthly data appear from 1997 only (when the government introduced them)
# So, this join won't work for rows before 1997
# I actually intend to do a SIC join later

# Getting all the CUSIP's that are potentially interlinked with date, ticker, siccd matching
tickers_potentially_linked_with_naics_date = finding_interlinked_cusips[((finding_interlinked_cusips['cusip_6digit_x']  != finding_interlinked_cusips['cusip_6digit_y']) &
                                                                       (~finding_interlinked_cusips['ticker_mod'].isnull()) &
                                                                       (~finding_interlinked_cusips['NAICS'].isnull()) & 
                                                                       (~finding_interlinked_cusips['Month'].isnull()) &
                                                                       (~finding_interlinked_cusips['Year'].isnull()))]

len(tickers_potentially_linked_with_naics_date['cusip_6digit_x'].unique())
# Wow, I found 100's of CUSIP's that are potentially linked with each other. That's something

187

In [48]:
# This time I will use ticker & sic code rather than ticker & naics
monthly_stock_data['SICCD'] = pd.to_numeric(monthly_stock_data['SICCD'] , errors = 'coerce').astype('float64')

finding_interlinked_cusips = pd.merge(monthly_stock_data[['Month','Year','SICCD','cusip_6digit','ticker_mod','COMNAM','Mcap']],
                                      comp_data[~comp_data['ticker_qtr'].isin(problematic_ticker_qtr)][
                                          ['Month','Year','sic_code','gvkey','cusip_6digit','ticker_mod','company_name','Mcap']],
                                      left_on = ['Month','Year','ticker_mod','SICCD'],
                                      right_on = ['Month','Year','ticker_mod','sic_code'],
                                      how = 'inner')

# Getting all the CUSIP's that are potentially interlinked with date, ticker, siccd matching
tickers_potentially_linked_with_sic_date = finding_interlinked_cusips[((finding_interlinked_cusips['cusip_6digit_x']  != finding_interlinked_cusips['cusip_6digit_y']) &
                                                                       (~finding_interlinked_cusips['ticker_mod'].isnull()) &
                                                                       (~finding_interlinked_cusips['SICCD'].isnull()) & 
                                                                       (~finding_interlinked_cusips['Month'].isnull()) &
                                                                       (~finding_interlinked_cusips['Year'].isnull()))]

len(tickers_potentially_linked_with_sic_date['cusip_6digit_x'].unique())
# Wow, I found 100's of CUSIP's that are potentially linked with each other. That's something

355

In [49]:
# What if I remove date from join? Exploring join on ticker & sector
# Trying to find matches on just ticker & sic code (I can try NAICS code next)

finding_interlinked_cusips = pd.merge(monthly_stock_data[['Month','Year','SICCD','cusip_6digit','ticker_mod','COMNAM','Mcap']],
                                      comp_data[~comp_data['ticker_qtr'].isin(problematic_ticker_qtr)][
                                          ['Month','Year','sic_code','gvkey','cusip_6digit','ticker_mod','company_name','Mcap']],
                                      left_on = ['ticker_mod','SICCD'],
                                      right_on = ['ticker_mod','sic_code'],
                                      how = 'inner')

# Getting all the CUSIP's that are potentially interlinked with ticker, siccd matching
tickers_potentially_linked_with_sic = finding_interlinked_cusips[((finding_interlinked_cusips['cusip_6digit_x']  != finding_interlinked_cusips['cusip_6digit_y']) &
                                                                  (~finding_interlinked_cusips['ticker_mod'].isnull()) &
                                                                  (~finding_interlinked_cusips['SICCD'].isnull()))]
len(tickers_potentially_linked_with_sic['cusip_6digit_x'].unique())
# Interestingly, found lot of additional matches when I don't match on date & year particuarly

799

In [50]:
# Let's take a closer look at all the additional matches that I got without date in join columns
tickers_potentially_linked_with_sic[~tickers_potentially_linked_with_sic['cusip_6digit_x'].isin(
    tickers_potentially_linked_with_sic_date['cusip_6digit_x'].unique())]

# Lot's of mismatches. Lot's of cases where 2 different companies (same SIC code) used the same ticker across different times
# I obviously expected some companies in same sector to grab an old ticker. But there are too many false positives

,Month_x,Year_x,SICCD,cusip_6digit_x,ticker_mod,COMNAM,Mcap_x,Month_y,Year_y,sic_code,gvkey,cusip_6digit_y,company_name,Mcap_y
11059,8,2005,3579.0,00081T,ABD,A C C O BRANDS CORP,1.368606e+09,3,1972,3579.0,3939,253034,DICK (A.B.) CO,NaN
11060,8,2005,3579.0,00081T,ABD,A C C O BRANDS CORP,1.368606e+09,6,1972,3579.0,3939,253034,DICK (A.B.) CO,NaN
11061,8,2005,3579.0,00081T,ABD,A C C O BRANDS CORP,1.368606e+09,9,1972,3579.0,3939,253034,DICK (A.B.) CO,NaN
11062,8,2005,3579.0,00081T,ABD,A C C O BRANDS CORP,1.368606e+09,12,1972,3579.0,3939,253034,DICK (A.B.) CO,NaN
11063,8,2005,3579.0,00081T,ABD,A C C O BRANDS CORP,1.368606e+09,3,1973,3579.0,3939,253034,DICK (A.B.) CO,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78445040,12,2020,1389.0,N22717,CLB,CORE LABORATORIES NV,1.179562e+09,9,1982,1389.0,3525,218677,CORE LABORATORIES,NaN
78445041,12,2020,1389.0,N22717,CLB,CORE LABORATORIES NV,1.179562e+09,12,1982,1389.0,3525,218677,CORE LABORATORIES,NaN
78445042,12,2020,1389.0,N22717,CLB,CORE LABORATORIES NV,1.179562e+09,3,1983,1389.0,3525,218677,CORE LABORATORIES,NaN
78445043,12,2020,1389.0,N22717,CLB,CORE LABORATORIES NV,1.179562e+09,6,1983,1389.0,3525,218677,CORE LABORATORIES,NaN


In [51]:
# Learnt date columns are too imp to remove
# But what if don't use any sector of company in matching?
# Trying to find matches on just ticker & date

finding_interlinked_cusips = pd.merge(monthly_stock_data[['Month','Year','SICCD','cusip_6digit','ticker_mod', 
                                                          'COMNAM','Mcap', 'ticker_qtr']],
                                      comp_data[~comp_data['ticker_qtr'].isin(problematic_ticker_qtr)][
                                          ['Month','Year','sic_code','gvkey','cusip_6digit','ticker_mod', 'company_name', 
                                           'Mcap']],
                                      left_on = ['Month','Year','ticker_mod'],
                                      right_on = ['Month','Year','ticker_mod'],
                                      how = 'inner')

# But before attempting look at matches, need to make certain adjustments keeping the following scenaris in mind
# Realized after few trails that these adjustments are imp when I am loosening join criteria by not using sector at all

# Scenario 1
# Let's say AGCO corporation used ticker 'AG' from 1980-2009 & ticker 'AGCO' from 2010-2020
# Also, let's say First majestic silver corporation also used ticker 'FMS' from 1980-2009 & changed to ticker 'AG' from 2010-2020
# The scenario I just described is problematic because AGCO corp 2005Q2 row will map to 2005 First majestic silver row
# (Remember: Compustat has only the most recent row. So all First Majestic Silver rows will have ticker 'AG', AGCO corporation will have 'AGCO')
# Ofcourse, if I had use SIC/NAICS, this problem wouldn't have happened because one is agricultural & another is mining sector

# Getting the full list of tickers that mapped to multiple company names (across time)
comp_ticker_mapping = pd.DataFrame(finding_interlinked_cusips.groupby('COMNAM')['ticker_mod'].nunique()).reset_index()
comp_ticker_mapping.columns = ['COMNAM', '#unique_ticker_mod']
problematic_comp = comp_ticker_mapping[comp_ticker_mapping['#unique_ticker_mod'] > 1]['COMNAM'].unique()

# Getting the full list of tickers that mapped to multiple CUSIP_6digits (across time)
cusip_ticker_mapping = pd.DataFrame(finding_interlinked_cusips.groupby('cusip_6digit_x')['ticker_mod'].nunique()).reset_index()
cusip_ticker_mapping.columns = ['cusip_6digit_x', '#unique_ticker_mod']
problematic_cusips = cusip_ticker_mapping[cusip_ticker_mapping['#unique_ticker_mod'] > 1]['cusip_6digit_x'].unique()

# Scenario 2: another scenario
# Let's say ABC construction corp was listed btw.1967-1980 and used the ticker 'ABC'
# Also, let's say abc broadcasting was listed from 1967 - 2020 but used the ticker 'AB' from 1967-1980 before moving to 'ABC'
# This scenario is turning out to be problematic because not all companies in CRSP are in compustat
# So now, ABC construction 1967 rows are getting attached to abc broadcasting rows since its the only company in compustat 1967
# Ofcourse, if I had use SIC/NAICS, this problem wouldn't have happened because one is real esatte & another is media sector

ticker_comp_mapping = pd.DataFrame(finding_interlinked_cusips.groupby('ticker_mod')['COMNAM'].nunique()).reset_index()
ticker_comp_mapping.columns = ['ticker_mod', '#unique_comp']
problematic_tickers_comp = ticker_comp_mapping[ticker_comp_mapping['#unique_comp'] > 1]['ticker_mod'].unique()

ticker_cusip_mapping = pd.DataFrame(finding_interlinked_cusips.groupby('ticker_mod')['cusip_6digit_x'].nunique()).reset_index()
ticker_cusip_mapping.columns = ['ticker_mod', '#unique_cusip']
problematic_ticker_cusip = ticker_cusip_mapping[ticker_cusip_mapping['#unique_cusip'] > 1]['ticker_mod'].unique()

# Compiling a full list of problematic Tickers
problematic_tickers = list(problematic_tickers_comp) + list(problematic_ticker_cusip)
problematic_tickers = list(set(problematic_tickers)) # Dropping duplicates from the list

# Getting all the CUSIP's that are potentially interlinked with ticker, siccd matching
tickers_potentially_linked_with_date = finding_interlinked_cusips[((finding_interlinked_cusips['cusip_6digit_x']  != finding_interlinked_cusips['cusip_6digit_y']) &
                                                                   (~finding_interlinked_cusips['ticker_mod'].isnull()) & 
                                                                   (~finding_interlinked_cusips['ticker_mod'].isin(problematic_tickers)) &
                                                                   (~finding_interlinked_cusips['Month'].isnull()) &
                                                                   (~finding_interlinked_cusips['Year'].isnull()) & 
                                                                   (~finding_interlinked_cusips['COMNAM'].isin(problematic_comp)) &
                                                                   (~finding_interlinked_cusips['cusip_6digit_x'].isin(problematic_cusips)))]

len(tickers_potentially_linked_with_date['cusip_6digit_x'].unique())
# lot's of matches OMG!
# Let's investigate the extra matches

468

In [52]:
# Let's take a closer look at all the extra matches over the initial set of matches that are settled on earlier(the 1st join)
cols_to_display = ['Month', 'Year', 'ticker_mod', 'gvkey', 'SICCD', 'sic_code', 'cusip_6digit_x', 'cusip_6digit_y', 'COMNAM', 
                  'company_name']
tickers_potentially_linked_with_date[~tickers_potentially_linked_with_date['cusip_6digit_x'].isin(
    tickers_potentially_linked_with_sic_date['cusip_6digit_x'].unique())][cols_to_display][21:30]

# Realized, getting so many additional matches because noticed that sic code isn't necessarily the same on both sides
# I didn't expect this
# Realied that govt. stopped issuing SIC numbers in 1997 (ever since NAICS was introduced)
# But private data companies like compustat, Dun & Bradstreet are still issuing their own SIC numbers (based on old system)
# So AMC entertainment SIC code is 7830 & 7832 in CRSP & Compustat respectively (because their SIC datasources are different)

# But despite multiple attempts, ticker mapping without using SIC as a joining column is proving too hard
# Although ticker makes more intuitive sense as an ID column than company name, there are too many false positives
# For example, I am joining APT Satellite holdings & ATS Andlauer income fund (2 companies with ticker in the same time)
# The real big problem is that there not all companies in CRSP are in Compustat
# So, if there's an 'abc' ticker on CRSP, and there happens to be an 'abc' ticker on compustat around the same time, it's joining
# However, by looking at names in output, I can tell they are 2 very different companies
# To summarize, if I want to use ticker, it has to be along with SIC matching

# May be I can escape using sic code if I try join on company names
# I am more hopeful about name matching.
# Afterall, how likely is it that 2 companies have the exact same name in the same period of time

,Month,Year,ticker_mod,gvkey,SICCD,sic_code,cusip_6digit_x,cusip_6digit_y,COMNAM,company_name
4555,12,2006,ATS,166254,4899.0,4210.0,00203R,00211C,A P T SATELLITE HOLDINGS LTD,ATS ANDLAUER INCOME FUND
4556,3,2007,ATS,166254,4899.0,4210.0,00203R,00211C,A P T SATELLITE HOLDINGS LTD,ATS ANDLAUER INCOME FUND
4557,6,2007,ATS,166254,4899.0,4210.0,00203R,00211C,A P T SATELLITE HOLDINGS LTD,ATS ANDLAUER INCOME FUND
4558,9,2007,ATS,166254,4899.0,4210.0,00203R,00211C,A P T SATELLITE HOLDINGS LTD,ATS ANDLAUER INCOME FUND
4559,12,2007,ATS,166254,4899.0,4210.0,00203R,00211C,A P T SATELLITE HOLDINGS LTD,ATS ANDLAUER INCOME FUND
4560,3,2008,ATS,166254,4899.0,4210.0,00203R,00211C,A P T SATELLITE HOLDINGS LTD,ATS ANDLAUER INCOME FUND
4561,6,2008,ATS,166254,4899.0,4210.0,00203R,00211C,A P T SATELLITE HOLDINGS LTD,ATS ANDLAUER INCOME FUND
5860,11,1996,ABDR,1895,7330.0,5812.0,002553,022909,ABACUS DIRECT CORP,AMBASSADOR FOOD SVCS CORP
5862,2,1997,ABDR,1895,7330.0,5812.0,002553,022909,ABACUS DIRECT CORP,AMBASSADOR FOOD SVCS CORP


In [53]:
# Using company name too to find any additional interlinked CUSIP's
# But company name need to be prepared properly before using it

# Taking only alphabets in companies name
monthly_stock_data['company_mod'] = monthly_stock_data['COMNAM'].str.replace('[^a-zA-Z]', '')
# Removing all white spaces (even between company name)
monthly_stock_data['company_mod'] = monthly_stock_data['company_mod'].str.replace(' ', '')
# Converting all characters to upper
monthly_stock_data['company_mod'] = monthly_stock_data['company_mod'].str.upper()

monthly_stock_data['company_mod'] = np.where(monthly_stock_data['COMNAM'].astype(str).str[-4:] == '-OLD', monthly_stock_data['company_mod'].astype(str).str[:-4], monthly_stock_data['company_mod'])
monthly_stock_data['company_mod'] = np.where(monthly_stock_data['COMNAM'].astype(str).str[-3:] == 'NEW', monthly_stock_data['company_mod'].astype(str).str[:-3], monthly_stock_data['company_mod'])
monthly_stock_data['company_mod'] = np.where(monthly_stock_data['COMNAM'].astype(str).str[-4:] == 'CL A', monthly_stock_data['company_mod'].astype(str).str[:-4], monthly_stock_data['company_mod'])
monthly_stock_data['company_mod'] = np.where(monthly_stock_data['COMNAM'].astype(str).str[-4:] == 'CL B', monthly_stock_data['company_mod'].astype(str).str[:-4], monthly_stock_data['company_mod'])
monthly_stock_data['company_mod'] = np.where(monthly_stock_data['COMNAM'].astype(str).str[-4:] == 'CL C', monthly_stock_data['company_mod'].astype(str).str[:-4], monthly_stock_data['company_mod'])
monthly_stock_data['company_mod'] = np.where(monthly_stock_data['COMNAM'].astype(str).str[-3:] == ' FD', monthly_stock_data['company_mod'].astype(str).str[:-3], monthly_stock_data['company_mod'])
monthly_stock_data['company_mod'] = np.where(monthly_stock_data['COMNAM'].astype(str).str[-3:] == ' TR', monthly_stock_data['company_mod'].astype(str).str[:-3], monthly_stock_data['company_mod'])
monthly_stock_data['company_mod'] = np.where(monthly_stock_data['COMNAM'].astype(str).str[-3:] == ' CO', monthly_stock_data['company_mod'].astype(str).str[:-3], monthly_stock_data['company_mod'])

monthly_stock_data['company_mod'] = np.where(monthly_stock_data['company_mod'].astype(str).str[-3:] == 'LTD', monthly_stock_data['company_mod'].astype(str).str[:-3], monthly_stock_data['company_mod'])
monthly_stock_data['company_mod'] = np.where(monthly_stock_data['company_mod'].astype(str).str[-3:] == 'INC', monthly_stock_data['company_mod'].astype(str).str[:-3], monthly_stock_data['company_mod'])
monthly_stock_data['company_mod'] = np.where(monthly_stock_data['company_mod'].astype(str).str[-3:] == 'ETF', monthly_stock_data['company_mod'].astype(str).str[:-3], monthly_stock_data['company_mod'])
monthly_stock_data['company_mod'] = np.where(monthly_stock_data['company_mod'].astype(str).str[-3:] == 'PLC', monthly_stock_data['company_mod'].astype(str).str[:-3], monthly_stock_data['company_mod'])
monthly_stock_data['company_mod'] = np.where(monthly_stock_data['company_mod'].astype(str).str[-4:] == 'CORP', monthly_stock_data['company_mod'].astype(str).str[:-4], monthly_stock_data['company_mod'])
monthly_stock_data['company_mod'] = np.where(monthly_stock_data['company_mod'].astype(str).str[-4:] == 'FUND', monthly_stock_data['company_mod'].astype(str).str[:-4], monthly_stock_data['company_mod'])
monthly_stock_data['company_mod'] = np.where(monthly_stock_data['company_mod'].astype(str).str[-5:] == 'COLTD', monthly_stock_data['company_mod'].astype(str).str[:-5], monthly_stock_data['company_mod'])
monthly_stock_data['company_mod'] = np.where(monthly_stock_data['company_mod'].astype(str).str[-5:] == 'TRUST', monthly_stock_data['company_mod'].astype(str).str[:-5], monthly_stock_data['company_mod'])
monthly_stock_data['company_mod'] = np.where(monthly_stock_data['company_mod'].astype(str).str[-6:] == 'BERHAD', monthly_stock_data['company_mod'].astype(str).str[:-6], monthly_stock_data['company_mod'])
monthly_stock_data['company_mod'] = np.where(monthly_stock_data['company_mod'].astype(str).str[-7:] == 'LIMITED', monthly_stock_data['company_mod'].astype(str).str[:-7], monthly_stock_data['company_mod'])
monthly_stock_data['company_mod'] = np.where(monthly_stock_data['company_mod'].astype(str).str[-7:] == 'COMPANY', monthly_stock_data['company_mod'].astype(str).str[:-7], monthly_stock_data['company_mod'])
monthly_stock_data['company_mod'] = np.where(monthly_stock_data['company_mod'].astype(str).str[-11:] == 'CORPORATION', monthly_stock_data['company_mod'].astype(str).str[:-11], monthly_stock_data['company_mod'])

In [54]:
# Similarly, I have to prepare company name in comp data

# Taking only alphabets in companies name
comp_data['company_mod'] = comp_data['company_name'].str.replace('[^a-zA-Z]', '')
# Removing all white spaces (even between company name)
comp_data['company_mod'] = comp_data['company_mod'].str.replace(' ', '')
# Converting all characters to upper
comp_data['company_mod'] = comp_data['company_mod'].str.upper()

comp_data['company_mod'] = np.where(comp_data['company_name'].astype(str).str[-4:] == '-OLD', comp_data['company_mod'].astype(str).str[:-4], comp_data['company_mod'])
comp_data['company_mod'] = np.where(comp_data['company_name'].astype(str).str[-3:] == 'NEW', comp_data['company_mod'].astype(str).str[:-3], comp_data['company_mod'])
comp_data['company_mod'] = np.where(comp_data['company_name'].astype(str).str[-4:] == 'CL A', comp_data['company_mod'].astype(str).str[:-4], comp_data['company_mod'])
comp_data['company_mod'] = np.where(comp_data['company_name'].astype(str).str[-4:] == 'CL B', comp_data['company_mod'].astype(str).str[:-4], comp_data['company_mod'])
comp_data['company_mod'] = np.where(comp_data['company_name'].astype(str).str[-4:] == 'CL C', comp_data['company_mod'].astype(str).str[:-4], comp_data['company_mod'])
comp_data['company_mod'] = np.where(comp_data['company_name'].astype(str).str[-3:] == ' FD', comp_data['company_mod'].astype(str).str[:-3], comp_data['company_mod'])
comp_data['company_mod'] = np.where(comp_data['company_name'].astype(str).str[-3:] == ' TR', comp_data['company_mod'].astype(str).str[:-3], comp_data['company_mod'])
comp_data['company_mod'] = np.where(comp_data['company_name'].astype(str).str[-3:] == ' CO', comp_data['company_mod'].astype(str).str[:-3], comp_data['company_mod'])

comp_data['company_mod'] = np.where(comp_data['company_mod'].astype(str).str[-3:] == 'LTD', comp_data['company_mod'].astype(str).str[:-3], comp_data['company_mod'])
comp_data['company_mod'] = np.where(comp_data['company_mod'].astype(str).str[-3:] == 'INC', comp_data['company_mod'].astype(str).str[:-3], comp_data['company_mod'])
comp_data['company_mod'] = np.where(comp_data['company_mod'].astype(str).str[-3:] == 'ETF', comp_data['company_mod'].astype(str).str[:-3], comp_data['company_mod'])
comp_data['company_mod'] = np.where(comp_data['company_mod'].astype(str).str[-3:] == 'PLC', comp_data['company_mod'].astype(str).str[:-3], comp_data['company_mod'])
comp_data['company_mod'] = np.where(comp_data['company_mod'].astype(str).str[-4:] == 'CORP', comp_data['company_mod'].astype(str).str[:-4], comp_data['company_mod'])
comp_data['company_mod'] = np.where(comp_data['company_mod'].astype(str).str[-4:] == 'FUND', comp_data['company_mod'].astype(str).str[:-4], comp_data['company_mod'])
comp_data['company_mod'] = np.where(comp_data['company_mod'].astype(str).str[-5:] == 'COLTD', comp_data['company_mod'].astype(str).str[:-5], comp_data['company_mod'])
comp_data['company_mod'] = np.where(comp_data['company_mod'].astype(str).str[-5:] == 'TRUST', comp_data['company_mod'].astype(str).str[:-5], comp_data['company_mod'])
comp_data['company_mod'] = np.where(comp_data['company_mod'].astype(str).str[-6:] == 'BERHAD', comp_data['company_mod'].astype(str).str[:-6], comp_data['company_mod'])
comp_data['company_mod'] = np.where(comp_data['company_mod'].astype(str).str[-7:] == 'LIMITED', comp_data['company_mod'].astype(str).str[:-7], comp_data['company_mod'])
comp_data['company_mod'] = np.where(comp_data['company_mod'].astype(str).str[-7:] == 'COMPANY', comp_data['company_mod'].astype(str).str[:-7], comp_data['company_mod'])
comp_data['company_mod'] = np.where(comp_data['company_mod'].astype(str).str[-11:] == 'CORPORATION', comp_data['company_mod'].astype(str).str[:-11], comp_data['company_mod'])

In [55]:
# Wondering, if it's possible for 1 company_mod to appear twice in each quarter
comp_data['company_qtr'] = comp_data['company_mod'].astype(str) + comp_data['Year'].astype(str) + comp_data['Month'].astype(str)
compustat_comp_qtr_repeat = pd.DataFrame(comp_data.groupby('company_mod')['cusip_6digit'].nunique()).reset_index()
compustat_comp_qtr_repeat.columns = ['company_qtr', '# rows']
print(compustat_comp_qtr_repeat['# rows'].value_counts())
compustat_comp_qtr_problematic = compustat_comp_qtr_repeat[compustat_comp_qtr_repeat['# rows'] > 1]['company_qtr']

# Creating similar company_mod_qtr column in CRSP data too
monthly_stock_data['company_qtr'] = monthly_stock_data['company_mod'].astype(str) + monthly_stock_data['Year'].astype(str) + monthly_stock_data['Month'].astype(str)
crsp_comp_qtr_repeat = pd.DataFrame(monthly_stock_data.groupby('company_qtr')['cusip_6digit'].nunique()).reset_index()
crsp_comp_qtr_repeat.columns = ['company_qtr', '# rows']
print(crsp_comp_qtr_repeat['# rows'].value_counts())
crsp_comp_qtr_problematic = crsp_comp_qtr_repeat[crsp_comp_qtr_repeat['# rows'] > 1]['company_qtr']

# Compiling a full list of problematic company qtr combinations
problematic_comp_mod_qtr = list(compustat_comp_qtr_problematic) + list(crsp_comp_qtr_problematic)

1     38729
2       199
3        12
4         2
10        1
Name: # rows, dtype: int64
1       3883429
2          8699
3           900
5           428
8           283
         ...   
83            1
104           1
82            1
80            1
2987          1
Name: # rows, Length: 108, dtype: int64


In [56]:
# I also checked if 1 company_mod can map to multiple gvkeys
comnam_gvkey_mapping = pd.DataFrame(comp_data.groupby('company_mod')['gvkey'].nunique()).reset_index()
comnam_gvkey_mapping.columns = ['company_mod','Num_unique_gvkey']
comnam_gvkey_mapping = comnam_gvkey_mapping.sort_values(by=['Num_unique_gvkey'],ascending = False)
comnam_gvkey_mapping[0:5]

# A lot of indices have this problem of 1 company_mod mapping to multiple gvkeys
# But there's few normal stocks that are using the same name
# There are 3 diff companies with the name "WEST")
# I don't like this situation. But I recognize that as long as these 'WEST's don't appear multiple times in same qtr it's not  a problem 
# Hence, I am not adding these names to problematic list

,company_mod,Num_unique_gvkey
29496,RBCTARGETCORPBD,10
19400,ISHARESIBONDSDECTRM,10
18690,INVESCOBULMUNBND,9
19640,ISHARESTARGETDATE,9
19395,ISHARESIBNDDECTR,8


In [57]:
# Not even going to use SIC code in join because it's ridiculously unlikely that 2 companies have literally the same name in same time
# Want to try to 2 joins
# 1) Joining on company_mod, Month, Year
# 2) Joining on company_mod only

# Exploring 1st join
finding_interlinked_cusips = pd.merge(monthly_stock_data[['cusip_6digit','Month', 'Year', 'company_mod', 'COMNAM']].drop_duplicates(),
                                      comp_data[~comp_data['company_mod'].isin(problematic_comp_mod_qtr)][
                                          ['cusip_6digit','gvkey','Month', 'Year', 'company_mod', 'company_name']].drop_duplicates(),
                                      left_on = ['Month', 'Year', 'company_mod'],
                                      right_on = ['Month', 'Year', 'company_mod'],
                                      how = 'inner')

# Getting all the CUSIP's that are potentially interlinked with date, company_name, siccd matching
company_names_potentially_linked_with_date = finding_interlinked_cusips[((finding_interlinked_cusips['cusip_6digit_x']  != finding_interlinked_cusips['cusip_6digit_y']) &
                                                                         (~finding_interlinked_cusips['company_mod'].isnull()) &
                                                                         (~finding_interlinked_cusips['Month'].isnull()) & 
                                                                         (~finding_interlinked_cusips['Year'].isnull()))]
len(company_names_potentially_linked_with_date['cusip_6digit_x'].unique())

853

In [58]:
finding_interlinked_cusips = pd.merge(monthly_stock_data[['cusip_6digit','company_mod','SICCD', 'Year',
                                                          'COMNAM']].drop_duplicates(),
                                      comp_data[~comp_data['company_mod'].isin(problematic_comp_mod_qtr)]
                                      [['cusip_6digit','gvkey','company_mod','company_name']].drop_duplicates(),
                                      left_on = ['company_mod'],
                                      right_on = ['company_mod'],
                                      how = 'inner')

# Getting all the CUSIP's that are potentially interlinked with company_name, siccd matching
company_names_potentially_linked = finding_interlinked_cusips[((finding_interlinked_cusips['cusip_6digit_x']  != finding_interlinked_cusips['cusip_6digit_y']) &
                                                                         (~finding_interlinked_cusips['company_mod'].isnull()))]
len(company_names_potentially_linked['cusip_6digit_x'].unique())

1140

In [59]:
# Let's take a closer look at the additional matches that I got without date in join columns
company_names_potentially_linked[~company_names_potentially_linked['cusip_6digit_x'].isin(
    company_names_potentially_linked_with_date['cusip_6digit_x'].unique())][1001:1010]

# There's some mismatches as expected
# For example, the company_mod AIRLEASE has been used by 2 different companies (unrelated) across time
# It's easy to think that mismatches are going only to be few (which company is going to use an old name? lol!)
# But found too many false positives
# There used to be a big company called Abington bank, based in Abington, MA in 1990. That company went bankrupt
# Another Abington bank from Abington, PA listed in 2000's
# The risk is too high for few hundred additional matches

# Code from investigation of a particular false positive
#monthly_stock_data[monthly_stock_data['company_mod'] == 'UNIONELECTRICCO'][['date', 'cusip_6digit', 'COMNAM', 'Mcap']]
#comp_data[comp_data['company_mod'] == 'UNIONELECTRICCO'][['quarter_end_date', 'cusip_6digit', 'company_name', 'Mcap','cusip']]

,cusip_6digit_x,company_mod,SICCD,Year,COMNAM,cusip_6digit_y,gvkey,company_name
66906,898402,FIRSTCAPITAL,6711.0,1989,FIRST CAPITAL CORP,31942S,117034,FIRST CAPITAL INC
66907,898402,FIRSTCAPITAL,6711.0,1990,FIRST CAPITAL CORP,31942S,117034,FIRST CAPITAL INC
67058,319846,FIRSTCONNECTICUTBAN,6711.0,1972,FIRST CONNECTICUT BANCORP INC,319850,186344,FIRST CONNECTICUT BANCORP
67059,319846,FIRSTCONNECTICUTBAN,6711.0,1973,FIRST CONNECTICUT BANCORP INC,319850,186344,FIRST CONNECTICUT BANCORP
67060,319846,FIRSTCONNECTICUTBAN,6711.0,1974,FIRST CONNECTICUT BANCORP INC,319850,186344,FIRST CONNECTICUT BANCORP
67061,319846,FIRSTCONNECTICUTBAN,6711.0,1975,FIRST CONNECTICUT BANCORP INC,319850,186344,FIRST CONNECTICUT BANCORP
67062,319846,FIRSTCONNECTICUTBAN,6711.0,1976,FIRST CONNECTICUT BANCORP INC,319850,186344,FIRST CONNECTICUT BANCORP
67063,319846,FIRSTCONNECTICUTBAN,6711.0,1977,FIRST CONNECTICUT BANCORP INC,319850,186344,FIRST CONNECTICUT BANCORP
67064,319846,FIRSTCONNECTICUTBAN,6711.0,1978,FIRST CONNECTICUT BANCORP INC,319850,186344,FIRST CONNECTICUT BANCORP


In [60]:
# I am wondering if I just should work with company,date matches or if there's any sense in using ticker, sic, date matches

tickers_potentially_linked_with_sic_date[~tickers_potentially_linked_with_sic_date['cusip_6digit_x'].isin(
    company_names_potentially_linked_with_date['cusip_6digit_x'].unique())][2500:2510]
# Explored the extra matches. Wow. There's some really good matches here
# For example, a company called Alfacell renamed to Tamir biotechnology. Ticker, SIC, date joined picked up this combination
# Picking up this sort of interlink is not even remotely possible if one relies only on company names
# Even if some sort of string matching, an interlink like Alfacell & Tamir biotechnology will never get picked up
# Another example is DNA MED INC renamed to DNA Medical

# Similarly, there's value in earlier discovered ticker,naics, date matches
tickers_potentially_linked_with_naics_date[~tickers_potentially_linked_with_naics_date['cusip_6digit_x'].isin(
    company_names_potentially_linked_with_date['cusip_6digit_x'].unique())]

,Month,Year,NAICS,cusip_6digit_x,ticker_mod,COMNAM,Mcap_x,gvkey,cusip_6digit_y,company_name,Mcap_y
3846,3,2014,325412.0,00972G,CLTX,CELSUS THERAPEUTICS PLC,2.669986e+07,16687,15119A,CELSUS THERAPEUTICS PLC -ADR,NaN
3847,6,2014,325412.0,00972G,CLTX,CELSUS THERAPEUTICS PLC,2.550680e+07,16687,15119A,CELSUS THERAPEUTICS PLC -ADR,NaN
3848,9,2014,325412.0,00972G,CLTX,CELSUS THERAPEUTICS PLC,2.509540e+07,16687,15119A,CELSUS THERAPEUTICS PLC -ADR,NaN
3849,12,2014,325412.0,00972G,CLTX,CELSUS THERAPEUTICS PLC,1.974720e+07,16687,15119A,CELSUS THERAPEUTICS PLC -ADR,NaN
3850,3,2015,325412.0,00972G,CLTX,CELSUS THERAPEUTICS PLC,3.126640e+06,16687,15119A,CELSUS THERAPEUTICS PLC -ADR,NaN
...,...,...,...,...,...,...,...,...,...,...,...
140023,12,2019,334112.0,G7945M,STX,SEAGATE TECHNOLOGY PLC,1.563130e+10,150937,G7997R,SEAGATE TECHNOLOGY HOLDINGS,15524.9780
140024,3,2020,334112.0,G7945M,STX,SEAGATE TECHNOLOGY PLC,1.273465e+10,150937,G7997R,SEAGATE TECHNOLOGY HOLDINGS,12558.7776
140025,6,2020,334112.0,G7945M,STX,SEAGATE TECHNOLOGY PLC,1.242322e+10,150937,G7997R,SEAGATE TECHNOLOGY HOLDINGS,12427.7668
140026,9,2020,334112.0,G7945M,STX,SEAGATE TECHNOLOGY PLC,1.263958e+10,150937,G7997R,SEAGATE TECHNOLOGY HOLDINGS,12692.1984


In [61]:
# To summarize, I will make effective use of 3 different matches methods to identify all potential matches
cusip_gvkey_mapping_discovered_matches = pd.concat([tickers_potentially_linked_with_naics_date[['cusip_6digit_x','gvkey','company_name']],
                                                   tickers_potentially_linked_with_sic_date[['cusip_6digit_x','gvkey','company_name']],
                                                   company_names_potentially_linked_with_date[['cusip_6digit_x','gvkey','company_name']]],
                                                   axis = 0)

# Removing duplicates
cusip_gvkey_mapping_discovered_matches = cusip_gvkey_mapping_discovered_matches.drop_duplicates(subset = ['cusip_6digit_x', 
                                                                                                         'gvkey'])

# Wondering if it's possible for 1 cusip_6digit_x to map to multiple gvkeys
cusip_6digit_x_gvkey_uniques = pd.DataFrame(cusip_gvkey_mapping_discovered_matches.groupby('cusip_6digit_x')['gvkey'].nunique()).reset_index()
cusip_6digit_x_gvkey_uniques.columns = ['cusip_6digit', '# Unique_gvkey']
print(cusip_6digit_x_gvkey_uniques['# Unique_gvkey'].value_counts())

# Discovered it's possible because even gvkey is not truly perfect identifier (as discussed in fundamental data prep code)
cusip_6digit_x_gvkey_uniques[cusip_6digit_x_gvkey_uniques['# Unique_gvkey'] == 2]
cusip_gvkey_mapping_discovered_matches[cusip_gvkey_mapping_discovered_matches['cusip_6digit_x'] == '12561E'][['gvkey','company_name']]
# For example, cusip_6digit_x '12561E' mapped to gvkey '147809' based on ticker, sic, month, year
# But the same cusip_6digit_x '12561E' also mapped to gvkey '6346' based on company_name, month, year

# If a cusip_6digit has 2 different gvkeys, let's just take the gvkey that appears in most number of rows (on Comp side)
num_of_gvkeys = pd.DataFrame(comp_data['gvkey'].value_counts()).reset_index()
num_of_gvkeys.columns = ['gvkey','Num_of_values']

cusip_gvkey_mapping_discovered_matches = pd.merge(cusip_gvkey_mapping_discovered_matches,
                                                  num_of_gvkeys,
                                                  left_on = 'gvkey',
                                                  right_on = 'gvkey',
                                                  how = 'left')
cusip_gvkey_mapping_discovered_matches = cusip_gvkey_mapping_discovered_matches.sort_values(by = ['cusip_6digit_x', 'gvkey', 
                                                                                                  'Num_of_values'],ascending = False)
cusip_gvkey_mapping_discovered_matches = cusip_gvkey_mapping_discovered_matches.drop_duplicates(subset = ['cusip_6digit_x'])
cusip_gvkey_mapping_discovered_matches = cusip_gvkey_mapping_discovered_matches[['cusip_6digit_x', 'gvkey']]
# Now all cusip_6digits map gvkey one to one

1    1112
2      23
Name: # Unique_gvkey, dtype: int64


In [62]:
# creating the cusip_6digit & gvkey mapping when cusip_6digits do match on CRSP & compustat side

cusip_gvkey_mapping_same_cusip = pd.merge(monthly_stock_data['cusip_6digit'].drop_duplicates(),
                                          comp_data[['cusip_6digit','gvkey']].drop_duplicates(),
                                          left_on = 'cusip_6digit',
                                          right_on = 'cusip_6digit',
                                          how = 'inner')
print(cusip_gvkey_mapping_same_cusip.shape)

# If a cusip_6digit maps to multiple gvkeys, let's just exclude it from matches (I can try cusip_8digit mapping for such cases)
cusip6digit_gvkey_uniques = pd.DataFrame(cusip_gvkey_mapping_same_cusip.groupby('cusip_6digit')['gvkey'].nunique()).reset_index()
cusip6digit_gvkey_uniques.columns = ['cusip_6digit', '# Unique gvkey']
print(cusip6digit_gvkey_uniques['# Unique gvkey'].value_counts()[0:5])
# known there like <1% cusip_6digits that are problematic

# Getting all such problematic cusip_6digits
problematic_id = cusip6digit_gvkey_uniques[cusip6digit_gvkey_uniques['# Unique gvkey'] > 1]['cusip_6digit']
cusip_gvkey_mapping_same_cusip = cusip_gvkey_mapping_same_cusip[~cusip_gvkey_mapping_same_cusip['cusip_6digit'].isin(
                                                                problematic_id)]

(27016, 2)
1    23747
2       82
3       23
4       18
5       16
Name: # Unique gvkey, dtype: int64


In [63]:
# If unable to match a cusip_6digit from above join (because 1 cusip_6digit mapping to multiple gvkeys, using cusip_6digit & date)
# Let's see how many additional matches I am able to get

cusip_gvkey_mapping_same_cusip_with_date = pd.merge(monthly_stock_data[~monthly_stock_data['cusip_6digit'].isin(
                                                    cusip_gvkey_mapping_same_cusip['cusip_6digit'])][
                                                    ['cusip_6digit','Month','Year','cusip_6digit_date']].drop_duplicates(),
                                                    comp_data[['cusip_6digit','gvkey','Month','Year']].drop_duplicates(),
                                                    left_on = ['cusip_6digit','Month','Year'],
                                                    right_on = ['cusip_6digit','Month','Year'],
                                                    how = 'inner')
print(cusip_gvkey_mapping_same_cusip_with_date.shape)

# If a cusip_6digit_date maps to multiple gvkeys, let's just exclude it from matches (I can try cusip_8digit mapping for such cases)
cusip6digit_gvkey_uniques = pd.DataFrame(cusip_gvkey_mapping_same_cusip_with_date.groupby('cusip_6digit_date')['gvkey'].nunique()).reset_index()
cusip6digit_gvkey_uniques.columns = ['cusip_6digit_date', '# Unique gvkey date']
print(cusip6digit_gvkey_uniques['# Unique gvkey date'].value_counts()[0:5])
# know there like <1% cusip_6digits that are problematic

# Getting all such problematic cusip_6digit_dates
problematic_id = cusip6digit_gvkey_uniques[cusip6digit_gvkey_uniques['# Unique gvkey date'] > 1]['cusip_6digit_date']
cusip_gvkey_mapping_same_cusip_with_date = cusip_gvkey_mapping_same_cusip_with_date[~cusip_gvkey_mapping_same_cusip_with_date
                                                                                    ['cusip_6digit_date'].isin(problematic_id)]
cusip_gvkey_mapping_same_cusip_with_date = cusip_gvkey_mapping_same_cusip_with_date.drop_duplicates(subset = ['cusip_6digit',
                                                                                                'gvkey','cusip_6digit_date'])
print(len(cusip_gvkey_mapping_same_cusip_with_date['cusip_6digit'].unique()))

# Didn't get many mappings this way. Just 3944 mappings
# Will not be using this data because very few additional mappings 

# Will only use mappings from cusip_gvkey_mapping_discovered_matches and cusip_gvkey_mapping_same_cusip

(80729, 5)
1    3944
2    2023
4     546
3     529
5     409
Name: # Unique gvkey date, dtype: int64
146


In [64]:
# Compiling a full list of cusip_6digit & gvkey matchings
cusip_gvkey_mapping_discovered_matches.rename(columns = {'cusip_6digit_x': 'cusip_6digit'}, inplace = True)
print(len(cusip_gvkey_mapping_discovered_matches[cusip_gvkey_mapping_discovered_matches['cusip_6digit'].isin(
                                           cusip_gvkey_mapping_same_cusip['cusip_6digit'])]))

# Realised that almost half of cusip_6digits in discovered matches are in cusip_gvkey_mapping_same_cusip
# This is partly because of 2 reasons:
# 1) Matching algorithm is not perfect. There are going to be few boundary cases that are wrong matches
# 2) Even compustat data is not perfect. For ex: amc has 2 gvkeys (1038 & 164271)
#    Ofcourse, GM doesn't have this problem as entire GM has just 1 gvkey
#    Matching algo is able to catch such mistakes in compustat data

# Lot of cases like amc have been found (companies which delisted and appeared again)
# But I am going to attempt to come up with some adjustments or create some new ID
# Because creating a new real unique ID is not worth it considering just few hundred cases (not much additional lift)
# Honestly, this whole additional company matching exercise is a massive time sink
# I worked so hard with ticker, company_name columns to find additional matches when CUSIP_6digits are not same
# Found around 1100, of which I could have mapped half anyway I just did cusip_6digit to cusip_6digit join (print line above)
# This whole matching excercise has been challenging! & fun(?) but time to move on

# Let's just rely on gvkey to be a unique company identifier
# Hence, I will prioritize matches that have been found with cusip_6 digit matching over matches when cusips are not matching 

cusip_gvkey_mapping_discovered_matches = cusip_gvkey_mapping_discovered_matches[~cusip_gvkey_mapping_discovered_matches['cusip_6digit'].isin(
                                                                                cusip_gvkey_mapping_same_cusip['cusip_6digit'])]


full_cusip6digit_gvkey = pd.concat([cusip_gvkey_mapping_same_cusip, cusip_gvkey_mapping_discovered_matches], axis = 0)

565


In [65]:
# Checking final version of cusip_6digit mapping file
print(full_cusip6digit_gvkey.shape)
print(len(full_cusip6digit_gvkey['cusip_6digit'].unique()))
print(pd.DataFrame(full_cusip6digit_gvkey.groupby('cusip_6digit')['gvkey'].nunique()).reset_index()['gvkey'].value_counts())

(24317, 2)
24317
1    24317
Name: gvkey, dtype: int64


In [66]:
# Joining gvkey into monthly_stock_data with cusip_6digit mapping file that was just created.
print(monthly_stock_data.shape)
print(full_cusip6digit_gvkey.shape)
monthly_stock_data = pd.merge(monthly_stock_data,
                              full_cusip6digit_gvkey,
                              left_on = 'cusip_6digit',
                              right_on = 'cusip_6digit',
                              how = 'left')
print(monthly_stock_data.shape)

(3938621, 34)
(24317, 2)
(3938621, 35)


In [67]:
print(monthly_stock_data.shape)
print(monthly_stock_data['gvkey'].isnull().sum())
print(len(monthly_stock_data['cusip_6digit'].unique()))
print(len(monthly_stock_data['cusip_8digit'].unique()))
print(len(monthly_stock_data[monthly_stock_data['gvkey'].isnull()]['cusip_6digit'].unique()))
# According to the observations there are few cusip_6digits of small companies which were on market for short period of time

(3938621, 35)
384282
29760
30265
5443


In [68]:
# Attempting to map on cusip_8digit

print(monthly_stock_data.shape)
print(comp_data.shape)
cusip8digit_gvkey_mapping_same_cusip = pd.merge(monthly_stock_data['cusip_8digit'].drop_duplicates(),
                                                comp_data[['cusip_8digit','gvkey']].drop_duplicates(),
                                                left_on = 'cusip_8digit',
                                                right_on = 'cusip_8digit',
                                                how = 'inner')
print(cusip8digit_gvkey_mapping_same_cusip.shape)

# As expected, no cusip_8digit maps to multiple gvkeys
# That's because on compustat side a cusip_8digit maps to a single gvkey
cusip_gvkey_comp_data_uniques = pd.DataFrame(cusip8digit_gvkey_mapping_same_cusip.groupby('cusip_8digit')['gvkey'].nunique()).reset_index()
cusip_gvkey_comp_data_uniques.columns = ['cusip_8digit', '# of gvkey']
cusip_gvkey_comp_data_uniques['# of gvkey'].value_counts()
# No need to exclude any matches

(3938621, 35)
(1828155, 72)
(23353, 2)


1    23353
Name: # of gvkey, dtype: int64

In [69]:
# Creating cusip_8digit mapping file and merging into monthly_stock_data

cusip_8digit_mapping_file = cusip8digit_gvkey_mapping_same_cusip[['cusip_8digit','gvkey']].drop_duplicates()

print(monthly_stock_data.shape)
monthly_stock_data = pd.merge(monthly_stock_data,
                              cusip_8digit_mapping_file,
                              left_on = 'cusip_8digit',
                              right_on = 'cusip_8digit',
                              how = 'left')
print(monthly_stock_data.shape)

(3938621, 35)
(3938621, 36)


In [70]:
print(monthly_stock_data.shape)
print(monthly_stock_data['gvkey_y'].isnull().sum())
print(len(monthly_stock_data['cusip_6digit'].unique()))
print(len(monthly_stock_data['cusip_8digit'].unique()))
print(len(monthly_stock_data[monthly_stock_data['gvkey_y'].isnull()]['cusip_8digit'].unique()))
# As expected cusip_8digits results in lesser matches

(3938621, 36)
589896
29760
30265
6912


In [71]:
# How many rows have gvkey_y but no gvkey_x (additional matches)

len(monthly_stock_data[(monthly_stock_data['gvkey_x'].isnull()) & (~monthly_stock_data['gvkey_y'].isnull())])

28402

In [72]:
# Just checking if there's any cases where gvkey_x doesn't match with gvkey_y

monthly_stock_data[((monthly_stock_data['gvkey_x'] != monthly_stock_data['gvkey_y']) &
                    (~monthly_stock_data['gvkey_x'].isnull()) &
                    (~monthly_stock_data['gvkey_y'].isnull()))]['SHRCD_Description'].value_counts()
# All cases where gvkey  from cusip_6digit mapping and cusip_8digit mapping are actually some index funds
# When both mappings don't match, let's consider the 8 digit mapping (very few mismatches)

ETF                991
Close end funds    197
Name: SHRCD_Description, dtype: int64

In [73]:
monthly_stock_data['gvkey'] =  monthly_stock_data['gvkey_y'].fillna(monthly_stock_data['gvkey_x'])
print(monthly_stock_data[['gvkey','gvkey_y','gvkey_x']].isnull().sum())
# Removing gvkey_x and gvkey_y. Don't need them anymore
monthly_stock_data = monthly_stock_data.drop(['gvkey_x','gvkey_y'],axis = 1)

gvkey      355880
gvkey_y    589896
gvkey_x    384282
dtype: int64


In [74]:
# Need to do few final checks
print(len(monthly_stock_data['gvkey'].unique()))
print(len(monthly_stock_data[monthly_stock_data['gvkey'].isnull()]['cusip_6digit'].unique()))
print(len(monthly_stock_data['cusip_6digit'].unique()))
# Why are counts not adding up perfectly?

# Is it possible for 1 cusip_6digit to map to multiple gvkey? I have encountered cases like this earlier
cusip_6digit_gvkey_mapping = pd.DataFrame(monthly_stock_data.groupby('cusip_6digit')['gvkey'].nunique()).reset_index()
cusip_6digit_gvkey_mapping.columns = ['cusip_6digit','Num_of_unique_gvkey']
cusip_mapto_multiple_gvkeys = cusip_6digit_gvkey_mapping[cusip_6digit_gvkey_mapping['Num_of_unique_gvkey'] > 1]['cusip_6digit']
print(monthly_stock_data[monthly_stock_data['cusip_6digit'].isin(cusip_mapto_multiple_gvkeys)]['SHRCD_Description'].value_counts())
# Once again, it's possible for some cusip_6digit's to map to multiple gvkeys. Some of these rows are normal stocks too!!!!
# At the stage of cusip_6digit join, excluded any cusip_6digit that maps to gvkeys
# However, it's possible that 1 cusip_6digit, can map to 2 different cusip_8digits (& 2 different gvkeys)
# Remember, there's a cusip_8digit join too along with cusip_6digit join

24524
5219
29760
ETF                  8229
Normal Stock         1778
Close end funds       301
Foreign Companies     254
REIT                   87
Name: SHRCD_Description, dtype: int64


In [75]:
# But, does one gvkey appear only once for a specific date (Month, year)?
# Otherwise, there's a problem with level of data

# Earlier, I made sure that level of data is cusip_6digit_date
monthly_stock_data['cusip_6digit_date'].value_counts().max()

# Testing gvkey, date as level of data
monthly_stock_data['gvkey_date'] = monthly_stock_data['gvkey'].astype(str) + monthly_stock_data['date'].astype(str)
print(monthly_stock_data['gvkey_date'].value_counts())
# Oh no! data is not unique at gvkey & date

# Some companies has listed their stock twice in market (at the same time)
# For example, Carnival has a US listed stock (Ticker CCL) & they have listed their UK ADR too (ticker CUK)
# This is not like classA & classB. Remember, I already took only 1 class for each company
# Literally same company. Listed effectively the same company twice. Crazy!!!
monthly_stock_data[monthly_stock_data['gvkey_date'] == '14162.01987-07-31'][['gvkey','date','cusip_6digit','TICKER','COMNAM',
                                                                            'SHRCLS', 'company_mod']]

# Very few such cases. Without investgating too much into it (because very few cases), let's take 1st cusip that appears
monthly_stock_data = monthly_stock_data.sort_values(by = ['gvkey', 'date', 'cusip_6digit'], ascending = True)

# Checking if gvkey's and cusip_6digit's match
print(monthly_stock_data[monthly_stock_data['gvkey'].isin(monthly_stock_data['cusip_6digit'])].shape)
# gvkeys and cusip 6 digit don't match
# So, let's create a new unique identifier called ultimate id
# when gvkey exists, ultimate id is gvkey. When gvkey is null, cusip 6digit becomes ultimate id
monthly_stock_data['ultimate_id'] = monthly_stock_data['gvkey'].fillna(monthly_stock_data['cusip_6digit'])

monthly_stock_data = monthly_stock_data.drop_duplicates(subset = ['ultimate_id', 'date'])

nan1973-01-31        1787
nan1973-02-28        1785
nan1973-03-30        1776
nan1972-12-29        1765
nan1972-11-30        1747
                     ... 
27980.01999-04-30       1
29123.01997-06-30       1
24723.01991-10-31       1
5530.02002-10-31        1
26930.02008-02-29       1
Name: gvkey_date, Length: 3580122, dtype: int64
(0, 36)


In [76]:
# Creating stock exchange by changing numerical codes
monthly_stock_data['stock_exchange'] = np.where(monthly_stock_data['PRIMEXCH'] == 'N', 'NYSE', None)
monthly_stock_data['stock_exchange'] = np.where(monthly_stock_data['PRIMEXCH'] == 'A', 'NYSE_MKT', monthly_stock_data['stock_exchange'])
monthly_stock_data['stock_exchange'] = np.where(monthly_stock_data['PRIMEXCH'] == 'Q', 'NASDAQ', monthly_stock_data['stock_exchange'])
monthly_stock_data['stock_exchange'] = np.where(monthly_stock_data['PRIMEXCH'] == 'R', 'ARCHA', monthly_stock_data['stock_exchange'])
monthly_stock_data['stock_exchange'] = np.where(monthly_stock_data['PRIMEXCH'] == 'B', 'BATS', monthly_stock_data['stock_exchange'])
monthly_stock_data['stock_exchange'] = np.where(monthly_stock_data['PRIMEXCH'] == 'X', 'OTHER', monthly_stock_data['stock_exchange'])

In [77]:
monthly_stock_data['ultimate_id_date'] = monthly_stock_data['ultimate_id'].astype(str) + monthly_stock_data['date'].astype(str)

print(monthly_stock_data.shape)
print(len(monthly_stock_data['ultimate_id_date'].unique()))
# The data is finally at ultimate id and date level

(3935354, 39)
3935354


### Selecting important columns only (initial cut)

In [78]:
# Selecting important columns only (initial cut)

# Keys and ids
vimp_ids = ['ultimate_id', 'date', 'ultimate_id_date','vwretd','ewretd','gvkey','cusip_6digit','cusip_8digit']
good_to_have_ids = ['COMNAM', 'CUSIP', 'NCUSIP']

# Predictors
vimp_preds = ['NAICS', 'PRC', 'VOL']
good_to_have_columns = ['SHROUT','stock_exchange','SHRCD_Description', 'Mcap']

# Target variable
target = ['RET']

monthly_stock_data = monthly_stock_data[vimp_ids + good_to_have_ids + vimp_preds + good_to_have_columns + target]

# ALso, let's not forget to dedup the data
monthly_stock_data = monthly_stock_data.drop_duplicates(subset = ['ultimate_id_date'])

In [79]:
# monthly_stock_data = monthly_stock_data.loc[:,~monthly_stock_data.columns.duplicated()]
# Can use this line if accidentally selected same column twice

### Merging Industry names based on NAICS codes

In [80]:
NAICS_2017_data = pd.read_excel(r'C:\Users\joshn\Downloads\MS in Data Science\Stock market analysis\NAICS codes\2017_NAICS_Structure.xlsx', header = None)
# Reading in excel using pd.read_excel
# header = None because the original file has no column names
# The excel columns has actually only 3 columns. 
# But pd.read_excel is creating 6 columns. So, just selecting the first 3 columns
NAICS_2017_data = NAICS_2017_data.iloc[:, 0:3]
NAICS_2017_data.columns = ['Change Indicator', 'Code', 'Industry_name']
NAICS_2017_data = NAICS_2017_data.iloc[3:]
NAICS_2017_data['Code'].value_counts().max() # Using value_counts().max(), observing that each NAICS Code appears only once
NAICS_2017_data.isnull().sum()

Change Indicator    2181
Code                  19
Industry_name         19
dtype: int64

In [81]:
# The change column is pretty useless. But there are few empty rows with both Code & Title missing
NAICS_2017_data = NAICS_2017_data.drop(['Change Indicator'],axis = 1)
# Removing the rows with null values in 'Code' or 'INDEX ITEM DESCRIPTION' - Dropna subset
NAICS_2017_data = NAICS_2017_data.dropna(subset = ['Code', 'Industry_name'])

In [82]:
# I need to merge NAICS codes from previous years too
NAICS_2012_data = pd.read_excel(r'C:\Users\joshn\Downloads\MS in Data Science\Stock market analysis\NAICS codes\2012_NAICS_Index_File.xlsx')
NAICS_2012_data = NAICS_2012_data.iloc[:, 0:2]
NAICS_2012_data.rename(columns = {'NAICS12':'Code', 'INDEX ITEM DESCRIPTION': 'Industry_name'}, inplace = True)
NAICS_2012_data = NAICS_2012_data.dropna(subset = ['Code', 'Industry_name'])

NAICS_2007_data = pd.read_excel(r'C:\Users\joshn\Downloads\MS in Data Science\Stock market analysis\NAICS codes\2007_NAICS_Index_File.xlsx')
NAICS_2007_data = NAICS_2007_data.iloc[:, 0:2]
NAICS_2007_data.rename(columns = {'NAICS07':'Code', 'INDEX ITEM DESCRIPTION': 'Industry_name'}, inplace = True)
NAICS_2007_data = NAICS_2007_data.dropna(subset = ['Code', 'Industry_name'])

NAICS_2002_data = pd.read_csv(r'C:\Users\joshn\Downloads\MS in Data Science\Stock market analysis\NAICS codes\2002_NAICS.txt', sep = '  ', delimiter = None)
NAICS_2002_data = NAICS_2002_data.iloc[:, 0:2]
NAICS_2002_data.rename(columns = {'Unnamed: 1': 'Industry_name'}, inplace = True)
# NAICS_2002_data = NAICS_2002_data.drop(['2002 NAICS Title'],axis = 1) 
# No need of this line because selected only first 3 columns when reading in csv
NAICS_2002_data = NAICS_2002_data.dropna(subset = ['Code', 'Industry_name'])

In [83]:
NAICS_2017_data['Year'] = 2017
NAICS_2012_data['Year'] = 2012
NAICS_2007_data['Year'] = 2007
NAICS_2002_data['Year'] = 2002

All_years_NAICS_combined = pd.concat([NAICS_2017_data,NAICS_2012_data,NAICS_2007_data,NAICS_2002_data], axis = 0)
All_years_NAICS_combined = All_years_NAICS_combined.sort_values(by=['Code','Year'],ascending = False)
All_years_NAICS_combined = All_years_NAICS_combined.drop_duplicates(subset = ['Code'],keep = "first")
All_years_NAICS_combined.isnull().sum()

Code             0
Industry_name    0
Year             0
dtype: int64

In [84]:
# Need to make sure that the format of join keys are the same on both sides
monthly_stock_data['NAICS_str'] = monthly_stock_data['NAICS'].astype(str)
All_years_NAICS_combined['Code'] = All_years_NAICS_combined['Code'].astype(str) + '.0'

monthly_stock_data = pd.merge(monthly_stock_data,
                              All_years_NAICS_combined[['Code','Industry_name']],
                              left_on = 'NAICS_str',
                              right_on = 'Code',
                              how = 'left')

# Checking if there are any NAICS codes which don't have a match on the right side data
monthly_stock_data['Industry_name'].isnull().sum()

2794855

In [85]:
# Checking which NAICS codes don't have a match in our NAICS industry file
monthly_stock_data[((monthly_stock_data['Industry_name'].isnull()) & ~(monthly_stock_data['NAICS'].isnull()))]['NAICS'].value_counts()

514191.0    1414
513310.0    1115
452110.0     906
421690.0     862
233210.0     861
            ... 
331942.0       7
332110.0       6
514519.0       5
235930.0       3
481300.0       1
Name: NAICS, Length: 99, dtype: int64

### Creating derived variables

In [86]:
# Let's create prior 3 month % change for every row
# But before I do that, I need to sort the dataframe on Permno & date columns

# Realized that date is not in datetime format
monthly_stock_data['date'].dtype # It's not datetime

dtype('<M8[ns]')

In [87]:
monthly_stock_data['date'] = pd.to_datetime(monthly_stock_data['date']) #Converting date to datetime format
# monthly_stock_data['date'].dt.month.value_counts() # Just to make sure datetime code has worked

### Let's create prior 1M, 2M, 3M & 6M % change for every row

In [88]:
monthly_stock_data = monthly_stock_data.sort_values(by=['ultimate_id','date'],ascending = True)

# Using shift to shift the return column
monthly_stock_data['RET'] = pd.to_numeric(monthly_stock_data['RET'], errors = 'coerce')
monthly_stock_data['RET_shift1'] = monthly_stock_data['RET'].shift(1).astype('float64')
monthly_stock_data['RET_shift2'] = monthly_stock_data['RET'].shift(2).astype('float64')
monthly_stock_data['RET_shift3'] = monthly_stock_data['RET'].shift(3).astype('float64')
monthly_stock_data['RET_shift4'] = monthly_stock_data['RET'].shift(4).astype('float64')
monthly_stock_data['RET_shift5'] = monthly_stock_data['RET'].shift(5).astype('float64')
monthly_stock_data['RET_shift6'] = monthly_stock_data['RET'].shift(6).astype('float64')

monthly_stock_data['1M_RET'] = (1+monthly_stock_data['RET_shift1'])-1
monthly_stock_data['2M_RET'] = (1+monthly_stock_data['RET_shift1'])*(1+monthly_stock_data['RET_shift2'])-1
monthly_stock_data['3M_RET'] = (1+monthly_stock_data['RET_shift1'])*(1+monthly_stock_data['RET_shift2'])*(1+monthly_stock_data['RET_shift3'])-1
monthly_stock_data['6M_RET'] = (1+monthly_stock_data['RET_shift1'])*(1+monthly_stock_data['RET_shift2'])*(1+monthly_stock_data['RET_shift3'])*(1+monthly_stock_data['RET_shift4'])*(1+monthly_stock_data['RET_shift5'])*(1+monthly_stock_data['RET_shift6'])-1

# But I also need to make sure that I am not using some different stock returns to calculate prior returns for every row
monthly_stock_data['ultimate_id_shift1'] = monthly_stock_data['ultimate_id'].shift(1)
monthly_stock_data['ultimate_id_shift2'] = monthly_stock_data['ultimate_id'].shift(2)
monthly_stock_data['ultimate_id_shift3'] = monthly_stock_data['ultimate_id'].shift(3)
monthly_stock_data['ultimate_id_shift6'] = monthly_stock_data['ultimate_id'].shift(6)

monthly_stock_data['gap_date_shift1'] = (monthly_stock_data['date'] - monthly_stock_data['date'].shift(1)).dt.days
monthly_stock_data['gap_date_shift2'] = (monthly_stock_data['date'] - monthly_stock_data['date'].shift(2)).dt.days
monthly_stock_data['gap_date_shift3'] = (monthly_stock_data['date'] - monthly_stock_data['date'].shift(3)).dt.days
monthly_stock_data['gap_date_shift6'] = (monthly_stock_data['date'] - monthly_stock_data['date'].shift(6)).dt.days


monthly_stock_data['1M_RET'] = np.where(((monthly_stock_data['ultimate_id'] == monthly_stock_data['ultimate_id_shift1']) &
                                        (monthly_stock_data['gap_date_shift1'].between(25, 35, inclusive=False))) , 
                                         monthly_stock_data['1M_RET'], None)
monthly_stock_data['2M_RET'] = np.where(((monthly_stock_data['ultimate_id'] == monthly_stock_data['ultimate_id_shift2']) &
                                        (monthly_stock_data['gap_date_shift2'].between(53, 67, inclusive=False))) , 
                                         monthly_stock_data['2M_RET'], None)
monthly_stock_data['3M_RET'] = np.where(((monthly_stock_data['ultimate_id'] == monthly_stock_data['ultimate_id_shift3']) &
                                        (monthly_stock_data['gap_date_shift3'].between(83, 97, inclusive=False))) , 
                                         monthly_stock_data['3M_RET'], None)
# don't need the code below because the dataframe is sorted by Permno & date
#monthly_stock_data['3M_RET'] = np.where(((monthly_stock_data['ultimate_id'] == monthly_stock_data['ultimate_id_shift1']) & 
#                                          (monthly_stock_data['ultimate_id'] == monthly_stock_data['ultimate_id_shift2']) & 
#                                          (monthly_stock_data['ultimate_id'] == monthly_stock_data['ultimate_id_shift3'])), 
#                                         monthly_stock_data['3M_RET'], None)
monthly_stock_data['6M_RET'] = np.where(((monthly_stock_data['ultimate_id'] == monthly_stock_data['ultimate_id_shift6']) &
                                        (monthly_stock_data['gap_date_shift6'].between(170 ,190, inclusive=False))) , 
                                         monthly_stock_data['6M_RET'], None)

#monthly_stock_data[['ultimate_id','date','RET','RET_shift1','RET_shift2','RET_shift3','RET_shift4','RET_shift5','RET_shift6',
#                    '1M_RET', '2M_RET', '3M_RET', '6M_RET']].to_csv(r'C:\Users\joshn\Downloads\MS in Data Science\Stock market analysis\checking_all_months_return_code.csv',index = False)


### Let's create future 1M, 2M, 3M & 6M % change for every row (Potential target variable)

In [89]:
#Important points to remmember while cleaning final code
#All variable names have to be standardized
#concatinate code into relevant blocks(don't have too many blocks)
#make sure lines which need to be executed first or above
#please put heading for relevant sections(headings can be size 3 or size 6)

monthly_stock_data = monthly_stock_data.sort_values(by=['ultimate_id','date'],ascending = True)

monthly_stock_data['RET'] = pd.to_numeric(monthly_stock_data['RET'], errors = 'coerce')
monthly_stock_data['RET']= monthly_stock_data['RET'].astype('float64')
monthly_stock_data['RET_shiftnot1'] = monthly_stock_data['RET'].shift(-1)
monthly_stock_data['RET_shiftnot2'] = monthly_stock_data['RET'].shift(-2)
monthly_stock_data['RET_shiftnot3'] = monthly_stock_data['RET'].shift(-3)
monthly_stock_data['RET_shiftnot4'] = monthly_stock_data['RET'].shift(-4)
monthly_stock_data['RET_shiftnot5'] = monthly_stock_data['RET'].shift(-5)
monthly_stock_data['RET_shiftnot6'] = monthly_stock_data['RET'].shift(-6)

monthly_stock_data['ultimate_id_shiftnot1'] = monthly_stock_data['ultimate_id'].shift(-1)
monthly_stock_data['ultimate_id_shiftnot2'] = monthly_stock_data['ultimate_id'].shift(-2)
monthly_stock_data['ultimate_id_shiftnot5'] = monthly_stock_data['ultimate_id'].shift(-5)

monthly_stock_data['gap_date_shiftnot1'] = (monthly_stock_data['date'] - monthly_stock_data['date'].shift(-1)).dt.days
monthly_stock_data['gap_date_shiftnot2'] = (monthly_stock_data['date'] - monthly_stock_data['date'].shift(-2)).dt.days
monthly_stock_data['gap_date_shiftnot5'] = (monthly_stock_data['date'] - monthly_stock_data['date'].shift(-5)).dt.days


monthly_stock_data['1M_neg_RET'] = (1+monthly_stock_data['RET'])-1
monthly_stock_data['2M_neg_RET'] = (1+monthly_stock_data['1M_neg_RET'])*(1+monthly_stock_data['RET_shiftnot1'])-1
monthly_stock_data['3M_neg_RET'] = (1+monthly_stock_data['2M_neg_RET'])*(1+monthly_stock_data['RET_shiftnot2'])-1
monthly_stock_data['6M_neg_RET'] = (1+monthly_stock_data['3M_neg_RET'])*(1+monthly_stock_data['RET_shiftnot3'])*(1+monthly_stock_data['RET_shiftnot4'])*(1+monthly_stock_data['RET_shiftnot5'])-1

monthly_stock_data['1M_neg_RET'] = np.where((monthly_stock_data['ultimate_id'] == monthly_stock_data['ultimate_id']),
                                              monthly_stock_data['1M_neg_RET'], None)
monthly_stock_data['2M_neg_RET'] = np.where(((monthly_stock_data['ultimate_id'] == monthly_stock_data['ultimate_id_shiftnot1']) &
                                             (monthly_stock_data['gap_date_shiftnot1'].between(-35,-25, inclusive=False))), 
                                              monthly_stock_data['2M_neg_RET'], None)
monthly_stock_data['3M_neg_RET'] = np.where(((monthly_stock_data['ultimate_id'] == monthly_stock_data['ultimate_id_shiftnot2']) &
                                             (monthly_stock_data['gap_date_shiftnot2'].between(-65,-55, inclusive=False))), 
                                              monthly_stock_data['3M_neg_RET'], None)
monthly_stock_data['6M_neg_RET'] = np.where(((monthly_stock_data['ultimate_id'] == monthly_stock_data['ultimate_id_shiftnot5']) &
                                             (monthly_stock_data['gap_date_shiftnot5'].between(-160,-140, inclusive=False))), 
                                              monthly_stock_data['6M_neg_RET'], None)

#monthly_stock_data[['ultimate_id','date','RET','RET_shiftnot1','RET_shiftnot2','RET_shiftnot3','RET_shiftnot4','RET_shiftnot5','RET_shiftnot6',
#                     '1M_neg_RET','2M_neg_RET', '3M_neg_RET', '6M_neg_RET']].to_csv(r'C:\Users\joshn\Downloads\MS in Data Science\Stock market analysis\monthly_fututre_data.csv',index = False)


### Creating 1M,2M,3M and 6M vwretd and ewretd(both prev. and future) using RET

In [90]:
index_data = monthly_stock_data[['date','vwretd','ewretd']].drop_duplicates(subset = 'date', keep = "last")        
index_data['vwretd_shift1'] = index_data['vwretd'].shift(1)
index_data['vwretd_shift2'] = index_data['vwretd'].shift(2)
index_data['vwretd_shift3'] = index_data['vwretd'].shift(3)
index_data['vwretd_shift4'] = index_data['vwretd'].shift(4)
index_data['vwretd_shift5'] = index_data['vwretd'].shift(5)
index_data['vwretd_shift6'] = index_data['vwretd'].shift(6)

index_data['vwretd_shiftnot1'] = index_data['vwretd'].shift(-1)
index_data['vwretd_shiftnot2'] = index_data['vwretd'].shift(-2)
index_data['vwretd_shiftnot3'] = index_data['vwretd'].shift(-3)
index_data['vwretd_shiftnot4'] = index_data['vwretd'].shift(-4)
index_data['vwretd_shiftnot5'] = index_data['vwretd'].shift(-5)
index_data['vwretd_shiftnot6'] = index_data['vwretd'].shift(-6)

index_data['ewretd_shift1'] = index_data['ewretd'].shift(1)
index_data['ewretd_shift2'] = index_data['ewretd'].shift(2)
index_data['ewretd_shift3'] = index_data['ewretd'].shift(3)
index_data['ewretd_shift4'] = index_data['ewretd'].shift(4)
index_data['ewretd_shift5'] = index_data['ewretd'].shift(5)
index_data['ewretd_shift6'] = index_data['ewretd'].shift(6)

index_data['ewretd_shiftnot1'] = index_data['ewretd'].shift(-1)
index_data['ewretd_shiftnot2'] = index_data['ewretd'].shift(-2)
index_data['ewretd_shiftnot3'] = index_data['ewretd'].shift(-3)
index_data['ewretd_shiftnot4'] = index_data['ewretd'].shift(-4)
index_data['ewretd_shiftnot5'] = index_data['ewretd'].shift(-5)
index_data['ewretd_shiftnot6'] = index_data['ewretd'].shift(-6)


index_data['vwretd_1M_prev'] = (1+index_data['vwretd_shift1'])-1
index_data['vwretd_2M_prev'] = (1+index_data['vwretd_1M_prev'])*(1+index_data['vwretd_shift2'])-1
index_data['vwretd_3M_prev'] = (1+index_data['vwretd_2M_prev'])*(1+index_data['vwretd_shift3'])-1
index_data['vwretd_6M_prev'] = (1+index_data['vwretd_3M_prev'])*(1+index_data['vwretd_shift4'])*(1+index_data['vwretd_shift5'])*(1+index_data['vwretd_shift6'])-1


index_data['vwretd_1M_futr'] = (1+index_data['vwretd'])-1
index_data['vwretd_2M_futr'] = (1+index_data['vwretd_1M_futr'])*(1+index_data['vwretd_shiftnot1'])-1
index_data['vwretd_3M_futr'] = (1+index_data['vwretd_2M_futr'])*(1+index_data['vwretd_shiftnot2'])-1
index_data['vwretd_6M_futr'] = (1+index_data['vwretd_3M_futr'])*(1+index_data['vwretd_shiftnot3'])*(1+index_data['vwretd_shiftnot4'])*(1+index_data['vwretd_shiftnot5'])-1

index_data['ewretd_1M_prev'] = (1+index_data['ewretd_shift1'])-1
index_data['ewretd_2M_prev'] = (1+index_data['ewretd_1M_prev'])*(1+index_data['ewretd_shift2'])-1
index_data['ewretd_3M_prev'] = (1+index_data['ewretd_2M_prev'])*(1+index_data['ewretd_shift3'])-1
index_data['ewretd_6M_prev'] = (1+index_data['ewretd_3M_prev'])*(1+index_data['ewretd_shift4'])*(1+index_data['ewretd_shift5'])*(1+index_data['ewretd_shift6'])-1


index_data['ewretd_1M_futr'] = (1+index_data['ewretd'])-1
index_data['ewretd_2M_futr'] = (1+index_data['ewretd_1M_futr'])*(1+index_data['ewretd_shiftnot1'])-1
index_data['ewretd_3M_futr'] = (1+index_data['ewretd_2M_futr'])*(1+index_data['ewretd_shiftnot2'])-1
index_data['ewretd_6M_futr'] = (1+index_data['ewretd_3M_futr'])*(1+index_data['ewretd_shiftnot3'])*(1+index_data['ewretd_shiftnot4'])*(1+index_data['ewretd_shiftnot5'])-1

print(monthly_stock_data.shape)
monthly_stock_data = pd.merge(monthly_stock_data,
                             index_data[['date','vwretd_1M_prev',
                                        'vwretd_2M_prev','vwretd_3M_prev','vwretd_6M_prev','vwretd_1M_futr','vwretd_2M_futr',
                                        'vwretd_3M_futr','vwretd_6M_futr','ewretd_1M_prev','ewretd_2M_prev','ewretd_3M_prev',
                                        'ewretd_6M_prev','ewretd_1M_futr','ewretd_2M_futr','ewretd_3M_futr','ewretd_6M_futr']],
                             left_on = 'date',
                             right_on = 'date',
                             how = 'left')
print(monthly_stock_data.shape)



monthly_stock_data['RET_vwretd_1M_prev'] = monthly_stock_data['1M_RET'] - monthly_stock_data['vwretd_1M_prev']
monthly_stock_data['RET_vwretd_2M_prev'] = monthly_stock_data['2M_RET'] - monthly_stock_data['vwretd_2M_prev']
monthly_stock_data['RET_vwretd_3M_prev'] = monthly_stock_data['3M_RET'] - monthly_stock_data['vwretd_3M_prev']
monthly_stock_data['RET_vwretd_6M_prev'] = monthly_stock_data['6M_RET'] - monthly_stock_data['vwretd_6M_prev']


monthly_stock_data['RET_vwretd_1M_futr'] = monthly_stock_data['1M_neg_RET'] - monthly_stock_data['vwretd_1M_futr']
monthly_stock_data['RET_vwretd_2M_futr'] = monthly_stock_data['2M_neg_RET'] - monthly_stock_data['vwretd_2M_futr']
monthly_stock_data['RET_vwretd_3M_futr'] = monthly_stock_data['3M_neg_RET'] - monthly_stock_data['vwretd_3M_futr']
monthly_stock_data['RET_vwretd_6M_futr'] = monthly_stock_data['6M_neg_RET'] - monthly_stock_data['vwretd_6M_futr']


monthly_stock_data['RET_ewretd_1M_prev'] = monthly_stock_data['1M_RET'] - monthly_stock_data['ewretd_1M_prev']
monthly_stock_data['RET_ewretd_2M_prev'] = monthly_stock_data['2M_RET'] - monthly_stock_data['ewretd_2M_prev']
monthly_stock_data['RET_ewretd_3M_prev'] = monthly_stock_data['3M_RET'] - monthly_stock_data['ewretd_3M_prev']
monthly_stock_data['RET_ewretd_6M_prev'] = monthly_stock_data['6M_RET'] - monthly_stock_data['ewretd_6M_prev']


monthly_stock_data['RET_ewretd_1M_futr'] = monthly_stock_data['1M_neg_RET'] - monthly_stock_data['ewretd_1M_futr']
monthly_stock_data['RET_ewretd_2M_futr'] = monthly_stock_data['2M_neg_RET'] - monthly_stock_data['ewretd_2M_futr']
monthly_stock_data['RET_ewretd_3M_futr'] = monthly_stock_data['3M_neg_RET'] - monthly_stock_data['ewretd_3M_futr']
monthly_stock_data['RET_ewretd_6M_futr'] = monthly_stock_data['6M_neg_RET'] - monthly_stock_data['ewretd_6M_futr']

# monthly_stock_data[0:1000].to_csv(r'C:\Users\joshn\Downloads\MS in Data Science\Stock market analysis\new file.csv')

(3935354, 56)
(3935354, 72)


### Creating volume relativity index 1M and 2M (measuring how big or small volume is relative to long term average)

In [91]:
monthly_stock_data = monthly_stock_data.sort_values(by=['ultimate_id','date'],ascending = True)
monthly_stock_data['Vol_shrout_shift1'] = (monthly_stock_data['VOL']/monthly_stock_data['SHROUT']).shift(1)
monthly_stock_data['Vol_shrout_shift2'] = (monthly_stock_data['VOL']/monthly_stock_data['SHROUT']).shift(2)
monthly_stock_data['Vol_shrout_shift3'] = (monthly_stock_data['VOL']/monthly_stock_data['SHROUT']).shift(3)
monthly_stock_data['Vol_shrout_shift4'] = (monthly_stock_data['VOL']/monthly_stock_data['SHROUT']).shift(4)
monthly_stock_data['Vol_shrout_shift5'] = (monthly_stock_data['VOL']/monthly_stock_data['SHROUT']).shift(5)
monthly_stock_data['Vol_shrout_shift6'] = (monthly_stock_data['VOL']/monthly_stock_data['SHROUT']).shift(6)
monthly_stock_data['Vol_shrout_shift7'] = (monthly_stock_data['VOL']/monthly_stock_data['SHROUT']).shift(7)
monthly_stock_data['Vol_shrout_shift8'] = (monthly_stock_data['VOL']/monthly_stock_data['SHROUT']).shift(8)

monthly_stock_data['Vol_6M_avg'] = ((monthly_stock_data['Vol_shrout_shift2']+monthly_stock_data['Vol_shrout_shift3']+monthly_stock_data['Vol_shrout_shift4']+monthly_stock_data['Vol_shrout_shift5']+monthly_stock_data['Vol_shrout_shift6']+monthly_stock_data['Vol_shrout_shift7']))/6
monthly_stock_data['Vol_1M_6M_index'] = (monthly_stock_data['Vol_shrout_shift1']/monthly_stock_data['Vol_6M_avg'])

monthly_stock_data['ultimate_id_shift1'] = monthly_stock_data['ultimate_id'].shift(1)
monthly_stock_data['ultimate_id_shift7'] = monthly_stock_data['ultimate_id'].shift(7)

monthly_stock_data['gap_date_shift1'] = (monthly_stock_data['date'] - monthly_stock_data['date'].shift(1)).dt.days
monthly_stock_data['gap_date_shift7'] = (monthly_stock_data['date'] - monthly_stock_data['date'].shift(7)).dt.days


monthly_stock_data['Vol_1M_6M_index'] = np.where(((monthly_stock_data['ultimate_id'] == monthly_stock_data['ultimate_id_shift7']) &
                                                  (monthly_stock_data['gap_date_shift7'].between(200, 220, inclusive=False))), 
                                                   monthly_stock_data['Vol_1M_6M_index'], None)

#monthly_stock_data[['ultimate_id','ultimate_id_Vol_shift1','ultimate_id_Vol_shift7','date','VOL','Vol_shrout_shift1','Vol_shrout_shift2','Vol_shrout_shift3','Vol_shrout_shift4','Vol_shrout_shift5','Vol_shrout_shift6','Vol_shrout_shift7',
#                    'Vol_6M_avg', 'Vol_1M_index']].to_csv(r'C:\Users\joshn\Downloads\MS in Data Science\Stock market analysis\Vol_1M_check.csv',index = False)


In [92]:
monthly_stock_data['Vol_of_2M_6M_avg'] = (monthly_stock_data['Vol_shrout_shift3']+monthly_stock_data['Vol_shrout_shift4']+monthly_stock_data['Vol_shrout_shift5']+monthly_stock_data['Vol_shrout_shift6']+monthly_stock_data['Vol_shrout_shift7']+monthly_stock_data['Vol_shrout_shift8'])/6
monthly_stock_data['Vol_2M_avg'] = (monthly_stock_data['Vol_shrout_shift1']+monthly_stock_data['Vol_shrout_shift2'])/2
monthly_stock_data['Vol_2M_6M_index'] = monthly_stock_data['Vol_2M_avg']/monthly_stock_data['Vol_of_2M_6M_avg']

monthly_stock_data['ultimate_id_shift8'] = monthly_stock_data['ultimate_id'].shift(8)


monthly_stock_data['gap_date_shift8'] = (monthly_stock_data['date'] - monthly_stock_data['date'].shift(8)).dt.days


monthly_stock_data['Vol_2M_6M_index'] = np.where(((monthly_stock_data['ultimate_id'] == monthly_stock_data['ultimate_id_shift8']) &
                                                  (monthly_stock_data['gap_date_shift8'].between(230 ,250, inclusive=False))), 
                                                   monthly_stock_data['Vol_2M_6M_index'], None)

#monthly_stock_data[['ultimate_id','ultimate_id_Vol_2M_shift1','ultimate_id_Vol_2M_shift8','date','VOL','Vol_shrout_shift1','Vol_shrout_shift2','Vol_shrout_shift3','Vol_shrout_shift4','Vol_shrout_shift5','Vol_shrout_shift6','Vol_shrout_shift7','Vol_shrout_shift8',
#                    'Vol_of_2M_6M_avg', 'Vol_2M_avg','Vol_2M_index']].to_csv(r'C:\Users\joshn\Downloads\MS in Data Science\Stock market analysis\Vol_2M_check.csv',index = False)

### Calculating volume of shares 6M average for a task in a daily data

In [93]:
# First let's get prior 6M vol avg from monthly data
# For all jan rows in daily data, I will merge Jun-Dec 6M vol average

monthly_stock_data = monthly_stock_data.sort_values(by=['ultimate_id','date'],ascending = True)
monthly_stock_data['Vol_shrout_6M_avg'] = (monthly_stock_data['Vol_shrout_shift1']+monthly_stock_data['Vol_shrout_shift2']+monthly_stock_data['Vol_shrout_shift3']+monthly_stock_data['Vol_shrout_shift4']+monthly_stock_data['Vol_shrout_shift5']+monthly_stock_data['Vol_shrout_shift6'])/6
monthly_stock_data['ultimate_id_shift6'] = monthly_stock_data['ultimate_id'].shift(6)
monthly_stock_data['gap_date_shift6'] = (monthly_stock_data['date'] - monthly_stock_data['date'].shift(6)).dt.days
monthly_stock_data['Vol_shrout_6M_avg'] = np.where(((monthly_stock_data['ultimate_id'] == monthly_stock_data['ultimate_id_shift6']) &
                                                    (monthly_stock_data['gap_date_shift6'].between(170 ,190, inclusive=False))), 
                                                   monthly_stock_data['Vol_shrout_6M_avg'], None)

# extracting month and year from date column because I need to join monthly and daily data using month and year as keys.
monthly_stock_data['date'] = pd.to_datetime(monthly_stock_data['date']) 
monthly_stock_data['Month'] = monthly_stock_data['date'].dt.month
monthly_stock_data['Year'] = monthly_stock_data['date'].dt.year

In [94]:
monthly_stock_data[['ultimate_id','gvkey','cusip_6digit','cusip_8digit','Month','Year',
                    'Vol_shrout_6M_avg']].to_csv(r'Vol_shrout_6M_avg_from monthly_data.csv', index = False)

### Joining Monthly stock data & Compustat data

In [95]:
# Looking at match rates between CRSP and Compustat data
monthly_stock_data['date'] = pd.to_datetime(monthly_stock_data['date'])
monthly_stock_data['Year'] = monthly_stock_data['date'].dt.year

# Getting number of rows by year in each dataframe
evaluating_cusip_merge = pd.DataFrame(monthly_stock_data[monthly_stock_data['SHRCD_Description'].isin(shrcd_to_consider)])['Year'].value_counts().reset_index()
evaluating_cusip_merge.columns = ['Year', 'Num_of_rows']

# Getting #rows where CUSIP 6digit's don't match (across all years in compustat data)
cusip_quarter = comp_data['cusip_6digit'].unique()
monthly_stock_data['comp_cusip_both_quarter_monthly'] = np.where(monthly_stock_data['cusip_6digit'].isin(cusip_quarter), 0, 1)
num_cusip_qrt_mnt = pd.DataFrame(monthly_stock_data[monthly_stock_data['SHRCD_Description'].isin(shrcd_to_consider)].groupby('Year')['comp_cusip_both_quarter_monthly'].sum()).reset_index()
num_cusip_qrt_mnt.columns = ['Year', 'Num_cusip6_no_match']

print(evaluating_cusip_merge.shape)
evaluating_cusip_merge = pd.merge(evaluating_cusip_merge,
                                  num_cusip_qrt_mnt,
                                  left_on = 'Year',
                                  right_on = 'Year',
                                  how = 'left')
print(evaluating_cusip_merge.shape)

# Getting #rows where CUSIP 8digit's don't match (across all years in compustat data)
cusip_quarter = comp_data['cusip_8digit'].unique()
monthly_stock_data['comp_cusip_both_quarter_monthly'] = np.where(monthly_stock_data['cusip_8digit'].isin(cusip_quarter), 0, 1)
num_cusip_qrt_mnt = pd.DataFrame(monthly_stock_data[monthly_stock_data['SHRCD_Description'].isin(shrcd_to_consider)].groupby('Year')['comp_cusip_both_quarter_monthly'].sum()).reset_index()
num_cusip_qrt_mnt.columns = ['Year', 'Num_cusip8_no_match']

print(evaluating_cusip_merge.shape)
evaluating_cusip_merge = pd.merge(evaluating_cusip_merge,
                                  num_cusip_qrt_mnt,
                                  left_on = 'Year',
                                  right_on = 'Year',
                                  how = 'left')
print(evaluating_cusip_merge.shape)

# Getting #rows where CUSIP 6 digit's don't match (across all years in compustat data)
comp_data['quarter_end_date'] = pd.to_datetime(comp_data['quarter_end_date'])
comp_data['calendar_year'] = comp_data['quarter_end_date'].dt.year

yearly_cusip_mismatch = pd.DataFrame(None) # better to have a null df, It will be easy to use during iterations.
for i in monthly_stock_data['Year'].unique():
    cusip_quarter_i = comp_data[comp_data['calendar_year'] == i]['cusip_6digit'].unique()
    yearly_mismatch_num = len(monthly_stock_data[(monthly_stock_data['Year'] == i) & 
                                                 (monthly_stock_data['SHRCD_Description'].isin(shrcd_to_consider)) &
                                                 (~monthly_stock_data['cusip_6digit'].isin(cusip_quarter_i))])
    num_df = pd.DataFrame([[i,yearly_mismatch_num]], columns=['Year', 'Num_cusip6_yearly_no_match'])
#    print(num_df)
    yearly_cusip_mismatch = pd.concat([yearly_cusip_mismatch , num_df], axis = 0)
    
print(evaluating_cusip_merge.shape)
evaluating_cusip_merge = pd.merge(evaluating_cusip_merge,
                                  yearly_cusip_mismatch,
                                  left_on = 'Year',
                                  right_on = 'Year',
                                  how = 'left')
print(evaluating_cusip_merge.shape)

# Getting #rows where CUSIP 8 digit's don't match (across all years in compustat data)
yearly_cusip_mismatch = pd.DataFrame(None) # better to have a null df, It will be easy to use during iterations.
for i in monthly_stock_data['Year'].unique():
    cusip_quarter_i = comp_data[comp_data['calendar_year'] == i]['cusip_8digit'].unique()
    yearly_mismatch_num = len(monthly_stock_data[(monthly_stock_data['Year'] == i) & 
                                                 (monthly_stock_data['SHRCD_Description'].isin(shrcd_to_consider)) &
                                                 (~monthly_stock_data['cusip_8digit'].isin(cusip_quarter_i))])
    num_df = pd.DataFrame([[i,yearly_mismatch_num]], columns=['Year', 'Num_cusip8_yearly_no_match'])
#    print(num_df)
    yearly_cusip_mismatch = pd.concat([yearly_cusip_mismatch , num_df], axis = 0)
    
print(evaluating_cusip_merge.shape)
evaluating_cusip_merge = pd.merge(evaluating_cusip_merge,
                                  yearly_cusip_mismatch,
                                  left_on = 'Year',
                                  right_on = 'Year',
                                  how = 'left')
print(evaluating_cusip_merge.shape)

evaluating_cusip_merge = evaluating_cusip_merge.sort_values(by=['Year'],ascending = True)

(54, 2)
(54, 3)
(54, 3)
(54, 4)
(54, 4)
(54, 5)
(54, 5)
(54, 6)


In [96]:
evaluating_cusip_merge

# Noticed the following trends
# 1) Using Cusip_6digit rather than Cusip_8 digit as a joining key results in additional matches
# 2) Interestingly, some CUSIP_6digits are not found in the specific year of CRSP data but in other years
# 3) mismatches generally peaked around 1970-1980
# 4) mismatches reduce gradually. Less than 5% of rows have no match in 2020

,Year,Num_of_rows,Num_cusip6_no_match,Num_cusip8_no_match,Num_cusip6_yearly_no_match,Num_cusip8_yearly_no_match
53,1967,26313,7307,9163,9865,11038
52,1968,26392,6063,7848,8046,9312
51,1969,27318,5263,7008,7275,8596
50,1970,28442,4907,6685,7032,8395
49,1971,29407,4880,6672,6973,8360
48,1972,36911,7821,9837,12136,13620
25,1973,66109,20830,23752,36573,38294
29,1974,62383,17363,20235,32692,34470
30,1975,60709,15565,18444,31056,32853
31,1976,60674,14849,17776,31189,32994


### Final grouping of monthly_stock_data columns before join

In [97]:
id_cols = ['ultimate_id','gvkey','CUSIP' ,'cusip_6digit','cusip_8digit','COMNAM','NAICS','Industry_name']

stock_info_cols = ['SHROUT','stock_exchange','Mcap']

time_cols = ['date','Month','Year']

price_cols = ['PRC','RET','RET_vwretd_1M_prev','RET_vwretd_2M_prev','RET_vwretd_3M_prev',
              'RET_vwretd_6M_prev','RET_vwretd_1M_futr','RET_vwretd_2M_futr','RET_vwretd_3M_futr','RET_vwretd_6M_futr']

vol_cols = ['VOL','Vol_1M_6M_index','Vol_2M_6M_index']

monthly_stock_data_imp_cols = monthly_stock_data[id_cols + stock_info_cols + time_cols + price_cols + vol_cols]

In [98]:
# Similarly selecting only the imp columns from comp data for join

comp_data_imp_cols = comp_data.drop(['cusip','ticker', 'split_adjusting_factor', 'company_name','reporting_frequency',
                                     'employees', 'sic_code', 'NAICS', 'Mcap','share_holder_equity', 
                                     'intangible_asset_total', 'asset_total', 'cusip_6digit', 'cusip_8digit','cusip_6digit_date', 
                                     'cusip_8digit_date', 'ticker_mod', 'ticker_qtr', 'company_mod', 'company_qtr', 
                                     'calendar_year', 'fiscal_quarter', 'fiscal_year', 'Month', 'Year'], axis = 1)

In [99]:
# Figuring out the closest result date for every row in monthly data

comp_data_imp_cols['result_reported_date_mod_month'] = pd.to_datetime(comp_data_imp_cols['result_reported_date_mod']).dt.month
comp_data_imp_cols['result_reported_date_mod_year'] = pd.to_datetime(comp_data_imp_cols['result_reported_date_mod']).dt.year

finding_closest_result_date = pd.merge(monthly_stock_data_imp_cols[['gvkey','date', 'Year']], 
                                       comp_data_imp_cols[['gvkey','result_reported_date_mod', 'result_reported_date_mod_year']],
                                       left_on = 'gvkey',
                                       right_on = 'gvkey',
                                       how = 'left')

finding_closest_result_date['Gap_btw_years'] = finding_closest_result_date['Year'] - finding_closest_result_date['result_reported_date_mod_year']

# The closest result reported date has to be either this year or the prev. year
finding_closest_result_date = finding_closest_result_date[(finding_closest_result_date['Gap_btw_years'] >= 0) & 
                                                          (finding_closest_result_date['Gap_btw_years'] <= 1)] 

finding_closest_result_date['date'] = pd.to_datetime(finding_closest_result_date['date'])
finding_closest_result_date['result_reported_date_mod'] = pd.to_datetime(finding_closest_result_date['result_reported_date_mod'])

finding_closest_result_date['gap_btw_days'] = finding_closest_result_date['date'] - finding_closest_result_date['result_reported_date_mod']
finding_closest_result_date = finding_closest_result_date[finding_closest_result_date['gap_btw_days'].dt.days > 30]

finding_closest_result_date = finding_closest_result_date.sort_values(by = ['gvkey','date','gap_btw_days'], 
                                                                      ascending = True).drop_duplicates(subset = ['gvkey','date'])

# Don't want to consider results that are more than a year old
finding_closest_result_date = finding_closest_result_date[finding_closest_result_date['gap_btw_days'].dt.days < 368]

In [100]:
# Merginng the closest result date to every row in monthly data (if there is a match)

print(monthly_stock_data_imp_cols.shape)
monthly_stock_data_imp_cols = pd.merge(monthly_stock_data_imp_cols,
                                       finding_closest_result_date[['gvkey', 'date', 'result_reported_date_mod']],
                                       left_on = ['gvkey', 'date'],
                                       right_on = ['gvkey', 'date'],
                                       how = 'left')
print(monthly_stock_data_imp_cols.shape)

(3935354, 27)
(3935354, 28)


In [101]:
comp_data_imp_cols['result_reported_date_mod'] = pd.to_datetime(comp_data_imp_cols['result_reported_date_mod'])

# Joining price data and fundemental data based on gvkey, result_reported_date

print(monthly_stock_data_imp_cols.shape)
monthly_stock_data_imp_cols = pd.merge(monthly_stock_data_imp_cols,
                                       comp_data_imp_cols,
                                       left_on = ['gvkey', 'result_reported_date_mod'],
                                       right_on = ['gvkey', 'result_reported_date_mod'],
                                       how = 'left')
print(monthly_stock_data_imp_cols.shape)

(3935354, 28)
(3939155, 76)


In [102]:
# Calculating commonly used financial ratios
monthly_stock_data_imp_cols['P/TB_ratio'] = monthly_stock_data_imp_cols['Mcap'] / monthly_stock_data_imp_cols['tangible_equity']
monthly_stock_data_imp_cols['P/TB_ratio'].replace([np.inf, -np.inf], np.nan,inplace=True)
monthly_stock_data_imp_cols['P/E_ratio'] = monthly_stock_data_imp_cols['Mcap'] / (monthly_stock_data_imp_cols['eps_core_excl_extr_12M'].fillna(monthly_stock_data_imp_cols['eps_12M'])*monthly_stock_data_imp_cols['num_shares_eps_12'])
monthly_stock_data_imp_cols['P/E_ratio'].replace([np.inf, -np.inf], np.nan,inplace=True)
monthly_stock_data_imp_cols['P/E_ratio_abs'] = monthly_stock_data_imp_cols['P/E_ratio'].abs()
monthly_stock_data_imp_cols['P/E_ratio_abs'].replace([np.inf, -np.inf], np.nan,inplace=True)
monthly_stock_data_imp_cols['P/S_ratio'] = monthly_stock_data_imp_cols['Mcap'] / monthly_stock_data_imp_cols['revenue_12M']
monthly_stock_data_imp_cols['P/S_ratio'].replace([np.inf, -np.inf], np.nan,inplace=True)

### Modeling

### Linear Regression

In [ ]:
monthly_stock_data_imp_cols.columns

# Taking a look at all the columns, derived variables once again

In [ ]:
selected_col = ['RET_vwretd_1M_prev','RET_vwretd_2M_prev', 'RET_vwretd_3M_prev', 'RET_vwretd_6M_prev','Vol_1M_6M_index', 
                'Vol_2M_6M_index','percentage_current_asset_intan', 'percentage_equity_intan','percentage_cash_st_intan', 
                'percentage_LT_debt_intan','percentage_current_asset_intan_1Y_change','percentage_equity_intan_1Y_change',
                'percentage_cash_st_intan_1Y_change','percentage_LT_debt_intan_1Y_change', 'major_acq_12M',
                'revenue_5Y_CAGR_adj_dilution_max1','revenue_3Y_CAGR_adj_dilution_max1','revenue_1Y_CAGR_adj_dilution_max1',
                'P/TB_ratio', 'P/E_ratio', 'P/S_ratio', 'P/E_ratio_abs']

# Modelling only for stocks with Mcap between 1Bn and 10Bn (small cap)
# Large caps move largely with index i think

X = monthly_stock_data_imp_cols[((monthly_stock_data_imp_cols['Mcap'] > 100000000) & 
                                 (monthly_stock_data_imp_cols['Mcap'] < 1000000000))][selected_col]

X.replace([np.inf, -np.inf], np.nan,inplace=True)

y = monthly_stock_data_imp_cols[((monthly_stock_data_imp_cols['Mcap'] > 100000000) & 
                                 (monthly_stock_data_imp_cols['Mcap'] < 1000000000))].RET_vwretd_1M_futr

y.replace([np.inf, -np.inf], np.nan,inplace=True)

In [ ]:
X.isnull().sum()
# There's significant % of nulls in few important columns

In [ ]:
# But this is a project just for my learning experience, I am going to impute all missing columns with mean
X.fillna(X.mean(), inplace=True)
y = y.fillna(y.mean())

In [ ]:
X.corr()

In [ ]:
# VIF check
pd.Series([variance_inflation_factor(X.values, i)  for i in range(X.shape[1])], index=X.columns)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2, random_state = 0)
print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)
print("y_train: ", y_train.shape)
print("y_test: ", y_test.shape)

In [ ]:
# Linear regression code

linear_model = LinearRegression()

X_train_scaled = preprocessing.scale(X_train)
y_train_scaled = preprocessing.scale(y_train) 
# Scaling target variable as well because there is a lot of variance in target variable too
# There are lot of video with 10 likes, 20 likes and there's video with 60,000 likes

X_test_scaled = preprocessing.scale(X_test)
y_test_scaled = preprocessing.scale(y_test)

linear_model.fit(X_train_scaled,y_train_scaled)
y_test_scaled_pred=linear_model.predict(X_test_scaled)
y_train_scaled_pred=linear_model.predict(X_train_scaled)

In [ ]:
# Test data metrics

print("R^2: ", round(r2_score(y_test,y_test_pred), 2))
print("Adjusted R^2: ", round(1 - ( 1-linear_model.score(X_test, y_test) ) * ( len(y) - 1 ) / ( len(y) - X.shape[1] - 1 ), 2))
print("RMSE: ", round(np.sqrt(mean_squared_error(y_test, y_test_pred)), 2))
print("MAE: ", round(mean_absolute_error(y_test,y_test_pred), 2))

In [ ]:
# Getting p values
linear_mod = sm.OLS(y_train,X_train)
fii = linear_mod.fit()
p_values = fii.summary2().tables[1]['P>|t|']
p_values
# Lot of variables are significant. Created useful derived variables
# Unfortunately, there is a lot of unexplained variance (stock markets are effected by numerous features)

In [ ]:
# Getting coefficients
coeff_df = pd.DataFrame(linear_model.coef_, X.columns, columns=['Coefficients'])
print(coeff_df)

In [ ]:
# Recursive feature elimination

model = LinearRegression()
rfe = RFE(model, 5)
fit = rfe.fit(X_train, y_train)

#print(fit.n_features_)
#print(fit.support_)
#print(fit.ranking_)

list(X_train.iloc[:, fit.support_].columns)

### Logistic Regression

In [ ]:
# creating categorical target variable for logistic regression
monthly_stock_data_imp_cols['RET_vwretd_1M_futr > 0'] = np.where(monthly_stock_data_imp_cols['RET_vwretd_1M_futr'] > 0, 1, 0)

In [ ]:
selected_col = ['RET_vwretd_1M_prev','RET_vwretd_2M_prev', 'RET_vwretd_3M_prev', 'RET_vwretd_6M_prev','Vol_1M_6M_index', 
                'Vol_2M_6M_index','percentage_current_asset_intan', 'percentage_equity_intan','percentage_cash_st_intan', 
                'percentage_LT_debt_intan','percentage_current_asset_intan_1Y_change','percentage_equity_intan_1Y_change',
                'percentage_cash_st_intan_1Y_change','percentage_LT_debt_intan_1Y_change', 'major_acq_12M',
                'revenue_5Y_CAGR_adj_dilution_max1','revenue_3Y_CAGR_adj_dilution_max1','revenue_1Y_CAGR_adj_dilution_max1',
                'P/TB_ratio', 'P/E_ratio', 'P/S_ratio', 'P/E_ratio_abs']

X = monthly_stock_data_imp_cols[((monthly_stock_data_imp_cols['Mcap'] > 100000000) & 
                                 (monthly_stock_data_imp_cols['Mcap'] < 1000000000))][selected_col]

X.replace([np.inf, -np.inf], np.nan,inplace=True)

y = monthly_stock_data_imp_cols[((monthly_stock_data_imp_cols['Mcap'] > 100000000) & 
                                 (monthly_stock_data_imp_cols['Mcap'] < 1000000000))]['RET_vwretd_1M_futr > 0']

y.replace([np.inf, -np.inf], np.nan,inplace=True)

X.fillna(X.mean(), inplace=True)
y = y.fillna(1) # Barely 100 nulls in target variable

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2, random_state = 0)
print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)
print("y_train: ", y_train.shape)
print("y_test: ", y_test.shape)

In [ ]:
# Building logistic regression

logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

y_test_pred = logistic_model.predict(X_test)
y_test_pred_probas = logistic_model.predict_proba(X_test)[:,1]

y_train_pred = logistic_model.predict(X_train)
y_train_pred_probas = logistic_model.predict_proba(X_train)[:,1]

In [ ]:
# Confusion matrix
print(metrics.confusion_matrix(y_test, y_test_pred))

In [ ]:
# Metrics
print(metrics.classification_report(y_test, y_test_pred))

In [ ]:
# ROC curve
fpr, tpr, _ = metrics.roc_curve(y_test,  y_test_pred_probas)
auc = metrics.roc_auc_score(y_test, y_test_pred_probas)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()

In [ ]:
# Variable importance in logistic regression

var_imp = pd.concat([pd.DataFrame(X.columns), pd.DataFrame(logistic_model.coef_).T], axis = 1)
var_imp.columns = ['var_name', 'coeff']

std_df = pd.DataFrame(np.std(X_train, 0)).reset_index()
std_df.columns = ['var_name', 'std']

var_imp = pd.merge(var_imp,
                   std_df,
                   left_on = 'var_name',
                   right_on = 'var_name',
                   how = 'left')
var_imp['coeff*std'] = var_imp['coeff']*var_imp['std']

var_imp['coeff*std_pos'] = abs(var_imp['coeff*std'])

var_imp = var_imp.sort_values(by = 'coeff*std_pos', ascending = False)
var_imp = var_imp.drop(['coeff*std_pos'], axis = 1)

var_imp['coeff'] = var_imp['coeff'].astype(str)
var_imp[0:5]

### Random Forest

In [ ]:
#from sklearn.ensemble import RandomForestRegressor
nEstimator = [100,200] # Reducing parameter for grid search
depth = [15,,30] # Too computation heavy for my laptop

RF = RandomForestRegressor()
hyperParam = [{'n_estimators':nEstimator,'max_depth': depth}]
gsv = GridSearchCV(RF,hyperParam,cv=5,verbose=1,scoring='r2',n_jobs=-1)

In [ ]:
selected_col = ['RET_vwretd_1M_prev','RET_vwretd_2M_prev', 'RET_vwretd_3M_prev', 'RET_vwretd_6M_prev','Vol_1M_6M_index', 
                'Vol_2M_6M_index','percentage_current_asset_intan', 'percentage_equity_intan','percentage_cash_st_intan', 
                'percentage_LT_debt_intan','percentage_current_asset_intan_1Y_change','percentage_equity_intan_1Y_change',
                'percentage_cash_st_intan_1Y_change','percentage_LT_debt_intan_1Y_change', 'major_acq_12M',
                'revenue_5Y_CAGR_adj_dilution_max1','revenue_3Y_CAGR_adj_dilution_max1','revenue_1Y_CAGR_adj_dilution_max1',
                'P/TB_ratio', 'P/E_ratio', 'P/S_ratio', 'P/E_ratio_abs']

# Modelling only for stocks with Mcap > 10 Bn
X = monthly_stock_data_imp_cols[((monthly_stock_data_imp_cols['Mcap'] > 100000000) & 
                                 (monthly_stock_data_imp_cols['Mcap'] < 1000000000))][selected_col]

X.replace([np.inf, -np.inf], np.nan,inplace=True)

y = monthly_stock_data_imp_cols[((monthly_stock_data_imp_cols['Mcap'] > 100000000) & 
                                 (monthly_stock_data_imp_cols['Mcap'] < 1000000000))].RET_vwretd_1M_futr

y.replace([np.inf, -np.inf], np.nan,inplace=True)

X.fillna(X.mean(), inplace=True)
y = y.fillna(y.mean())

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2, random_state = 0)
print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)
print("y_train: ", y_train.shape)
print("y_test: ", y_test.shape)

In [ ]:
gsv.fit(X_train, y_train)
print("Best HyperParameter: ",gsv.best_params_)
print(gsv.best_score_)
scores = gsv.cv_results_['mean_test_score'].reshape(len(nEstimator),len(depth))
plt.figure(figsize=(8, 8))
plt.subplots_adjust(left=.2, right=0.95, bottom=0.15, top=0.95)
plt.imshow(scores, interpolation='nearest', cmap=plt.cm.hot)
plt.xlabel('n_estimators')
plt.ylabel('max_depth')
plt.colorbar()
plt.xticks(np.arange(len(nEstimator)), nEstimator)
plt.yticks(np.arange(len(depth)), depth)
plt.title('Grid Search r^2 Score')
plt.show()
maxDepth=gsv.best_params_['max_depth']
nEstimators=gsv.best_params_['n_estimators']

In [ ]:
# Random forest model
rf_model = RandomForestRegressor(n_estimators = 100, max_depth = 15, max_features = 5)
rf_model.fit(X_train, y_train)

y_test_pred = rf_model.predict(X_test)
y_train_pred = rf_model.predict(X_train)

In [ ]:
print('Mean Absolute Error (MAE):', round(metrics.mean_absolute_error(y_test, y_test_pred), 2))
print('Median Absolute Error:', round(metrics.median_absolute_error(y_test, y_test_pred), 2))
print('Mean Absolute Percentage Error (MAPE):', round(metrics.mean_absolute_percentage_error(y_test, y_test_pred), 2))
print('Max Error:', round(metrics.max_error(y_test, y_test_pred), 2))
print('Mean Squared Error (MSE):', round(metrics.mean_squared_error(y_test, y_test_pred), 2))
print('Root Mean Squared Error (RMSE):', round(metrics.mean_squared_error(y_test, y_test_pred, squared=False), 2))
print('R^2:', round(metrics.r2_score(y_test, y_test_pred), 2))
print('Explained Variance Score:', round(metrics.explained_variance_score(y_test, y_test_pred), 2))

In [ ]:
rf_var_imp = pd.concat([pd.DataFrame(X.columns), pd.DataFrame(rf_model.feature_importances_)], axis = 1)
rf_var_imp.columns = ['var_name', '% imp']
rf_var_imp['% imp'] = rf_var_imp['% imp']*100
rf_var_imp = rf_var_imp.sort_values(by = ['% imp'], ascending = False)
rf_var_imp

### SVR

In [ ]:
# For SVR, before doing grid search, need to scale the data

X_train_scaled = preprocessing.scale(X_train)
y_train_scaled = preprocessing.scale(y_train) 
# Scaling target variable as well because there is a lot of variance in target variable too
# There are lot of video with 10 likes, 20 likes and there's video with 60,000 likes

X_test_scaled = preprocessing.scale(X_test)
y_test_scaled = preprocessing.scale(y_test)

In [ ]:
C_array = [0.1, 1, 5]
gamma = [0.001, 1]

svr_model = SVR()
hyperParam = [{'C':C_array,'gamma': gamma}]
gsv = GridSearchCV(svr_model,hyperParam,cv=5,verbose=1,scoring='r2',n_jobs=-1)

gsv.fit(X_train_scaled, y_train_scaled)
print("Best HyperParameter: ",gsv.best_params_)
print(gsv.best_score_)
scores = gsv.cv_results_['mean_test_score'].reshape(len(C_array),len(gamma))
plt.figure(figsize=(8, 8))
plt.subplots_adjust(left=.2, right=0.95, bottom=0.15, top=0.95)
plt.imshow(scores, interpolation='nearest', cmap=plt.cm.hot)
plt.xlabel('C')
plt.ylabel('gamma')
plt.colorbar()
plt.xticks(np.arange(len(C_array)), C_array)
plt.yticks(np.arange(len(gamma)), gamma)
plt.title('Grid Search r^2 Score')
plt.show()
gamma_para=gsv.best_params_['gamma']
C_para = gsv.best_params_['C']

In [ ]:
# Support Vector Machine

svm_model = SVR(kernel='linear', C=5,gamma=0.001)

svm_model.fit(X_train_scaled, y_train_scaled)

# make predictions
y_test_pred_scaled = svm_model.predict(X_test_scaled)
y_train_pred_scaled = svm_model.predict(X_train_scaled)

In [ ]:
print('R^2:', round(metrics.r2_score(y_test_scaled, y_test_pred_scaled), 2))
print('Explained Variance Score:', round(metrics.explained_variance_score(y_test_scaled, y_test_pred_scaled), 2))

### SVC

In [ ]:
selected_col = ['RET_vwretd_1M_prev','RET_vwretd_2M_prev', 'RET_vwretd_3M_prev', 'RET_vwretd_6M_prev','Vol_1M_6M_index', 
                'Vol_2M_6M_index','percentage_current_asset_intan', 'percentage_equity_intan','percentage_cash_st_intan', 
                'percentage_LT_debt_intan','percentage_current_asset_intan_1Y_change','percentage_equity_intan_1Y_change',
                'percentage_cash_st_intan_1Y_change','percentage_LT_debt_intan_1Y_change', 'major_acq_12M',
                'revenue_5Y_CAGR_adj_dilution_max1','revenue_3Y_CAGR_adj_dilution_max1','revenue_1Y_CAGR_adj_dilution_max1',
                'P/TB_ratio', 'P/E_ratio', 'P/S_ratio', 'P/E_ratio_abs']

X = monthly_stock_data_imp_cols[((monthly_stock_data_imp_cols['Mcap'] > 100000000) & 
                                 (monthly_stock_data_imp_cols['Mcap'] < 1000000000))][selected_col]

X.replace([np.inf, -np.inf], np.nan,inplace=True)

y = monthly_stock_data_imp_cols[((monthly_stock_data_imp_cols['Mcap'] > 100000000) & 
                                 (monthly_stock_data_imp_cols['Mcap'] < 1000000000))]['RET_vwretd_1M_futr > 0']

y.replace([np.inf, -np.inf], np.nan,inplace=True)

X.fillna(X.mean(), inplace=True)
y = y.fillna(1) # Barely 100 nulls in target variable

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2, random_state = 0)
print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)
print("y_train: ", y_train.shape)
print("y_test: ", y_test.shape)


# Scaling X_train & X_test
X_train_scaled = preprocessing.scale(X_train)
X_test_scaled = preprocessing.scale(X_test)

In [ ]:
C_array = [0.1, 1, 5]
gamma = [0.001, 1]

svc_model = SVC()
hyperParam = [{'C':C_array,'gamma': gamma}]
gsv = GridSearchCV(svc_model,hyperParam,cv=5,verbose=1,scoring='roc_auc',n_jobs=-1)

gsv.fit(X_train_scaled, y_train)
print("Best HyperParameter: ",gsv.best_params_)
print(gsv.best_score_)
scores = gsv.cv_results_['mean_test_score'].reshape(len(C_array),len(gamma))
plt.figure(figsize=(8, 8))
plt.subplots_adjust(left=.2, right=0.95, bottom=0.15, top=0.95)
plt.imshow(scores, interpolation='nearest', cmap=plt.cm.hot)
plt.xlabel('C')
plt.ylabel('gamma')
plt.colorbar()
plt.xticks(np.arange(len(C_array)), C_array)
plt.yticks(np.arange(len(gamma)), gamma)
plt.title('Grid Search AUC')
plt.show()
gamma_para=gsv.best_params_['gamma']
C_para = gsv.best_params_['C']

In [ ]:
# Building SVC model

svc_model = SVC(C = 5, gamma = 1, probability = True, kernel = 'linear')
svc_model.fit(X_train_scaled, y_train)

y_test_pred = svc_model.predict(X_test_scaled)
y_test_pred_probas = svc_model.predict_proba(X_test_scaled)[:,1]

y_train_pred = svc_model.predict(X_train_scaled)
y_train_pred_probas = svc_model.predict_proba(X_train_scaled)[:,1]

In [ ]:
# Confusion matrix
print(metrics.confusion_matrix(y_test, y_test_pred))

In [ ]:
# Metrics
print(metrics.classification_report(y_test, y_test_pred))

In [ ]:
# ROC curve
fpr, tpr, _ = metrics.roc_curve(y_test,  y_test_pred_probas)
auc = metrics.roc_auc_score(y_test, y_test_pred_probas)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()

In [ ]:
# Variable importance in SVM

var_imp = pd.concat([pd.DataFrame(X.columns), pd.DataFrame(svc_model.coef_).T], axis = 1)
var_imp.columns = ['var_name', 'coeff']
var_imp['coeff_pos'] = abs(var_imp['coeff'])
var_imp = var_imp.sort_values(by = 'coeff_pos', ascending = False)
var_imp = var_imp.drop(['coeff_pos'], axis = 1)

var_imp['coeff'] = var_imp['coeff'].astype(str)
var_imp[0:5]